In [30]:
import pubchempy as pcp

def smiles_to_casrn(smiles):
    try:
        compounds = pcp.get_compounds(smiles, 'smiles')
        if compounds:
            return compounds[0]
    except Exception as e:
        print(f"Error: {e}")
        return None

smiles_string = "FC1(F)CCC(C(O)=O)CC1"
print(smiles_to_casrn(smiles_string))


Compound(2779136)


In [ ]:
import pubchempy as pcp

def smiles_to_casrn(smiles):
    compounds = pcp.get_compounds(smiles, 'smiles')
    if compounds:
        return compounds[0]

smiles_string = "FC1(F)CCC(C(O)=O)CC1"
print(smiles_to_casrn(smiles_string))


In [27]:
from openbabel import openbabel

def smiles_to_ob_molecule(smiles):
    obConversion = openbabel.OBConversion()
    obConversion.SetInAndOutFormats("smi", "mol")

    mol = openbabel.OBMol()
    obConversion.ReadString(mol, smiles)

    return mol

# Test with a SMILES string
smiles_string = "CC(=O)OC1=CC=CC=C1C(=O)O"  # SMILES for Aspirin
mol = smiles_to_ob_molecule(smiles_string)

In [7]:
import sys
import os
import math
import pandas as pd
from rdkit import Chem
from openbabel import openbabel
from openbabel import pybel

from rdkit import Chem
from rdkit.Chem import rdForceFieldHelpers
from rdkit.Chem import AllChem, Descriptors
import py3Dmol
import numpy as np

def find_conf_with_min_energy(mol):
    optimization_result = rdForceFieldHelpers.MMFFOptimizeMoleculeConfs(mol)
    
    min_energy, min_energy_conf_index = pow(10,5), None
    for index, (status, energy) in enumerate(optimization_result):
        if energy < min_energy and status == 0:
            min_energy_conf_index = index
            min_energy = min(min_energy, energy)

    return min_energy_conf_index, min_energy, mol

def rdkit_to_openbabel(rdkit_mol):
    sdf_writer = Chem.SDWriter('temp.sdf')
    sdf_writer.write(rdkit_mol)
    sdf_writer.close()

    ob_mol = next(pybel.readfile('sdf', 'temp.sdf'))

    return ob_mol.OBMol


EXCEL_PATH = r'C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part\molecule_features\pKa_Predicition_Starting data_2024.01.19.xlsx'

df = pd.read_excel(EXCEL_PATH, sheet_name="Main_List")

for index, row in df.iterrows():

    if pd.isnull(row['Smiles']):
        continue
    
    smiles_string = row['Smiles']
    mol = Chem.MolFromSmiles(smiles_string)

    # mol = Chem.MolFromSmiles('FC(F)(C1(NC2)CC2C1)F')
    mol = Chem.AddHs(mol)
    rdForceFieldHelpers.MMFFSanitizeMolecule(mol)

    num_rotatable_bonds = Descriptors.NumRotatableBonds(mol)
    amount_of_confs = pow(3, num_rotatable_bonds + 3)
    AllChem.EmbedMultipleConfs(mol, numConfs=amount_of_confs, randomSeed=3407)
    # for atom in mol.GetAtoms():
    #     atom.SetAtomMapNum(atom.GetIdx())

    ri = mol.GetRingInfo()

    min_energy_conf_index, min_energy, mol = find_conf_with_min_energy(mol)

    mol = rdkit_to_openbabel(mol)

    # You can get a list of charge models
    #  using `obabel -L charges`
    methods = [ 'mmff94', 'gasteiger', 'eem2015bm' ]

    print(f"SMILES: {smiles_string}")
    for method in methods:

        cm = openbabel.OBChargeModel.FindType(method)
        cm.ComputeCharges(mol)
        # but in C++ or Python we can get the dipole moment vector
        dipole = cm.GetDipoleMoment(mol)
        moment = math.sqrt(dipole.GetX()**2+dipole.GetY()**2+dipole.GetZ()**2)

        print(method, moment)

SMILES: FC1(F)CCC(C(O)=O)CC1
mmff94 0.0
gasteiger 1.8602753506970129
eem2015bm 0.0
SMILES: OC(C1CCCCC1)=O
mmff94 0.0
gasteiger 1.9152180364077103
eem2015bm 0.0
SMILES: OC([C@@H]1C[C@H]2C[C@H]2C1)=O
mmff94 0.0
gasteiger 1.8883502568360393
eem2015bm 0.0
SMILES: OC([C@H]1C[C@H]2C[C@H]2C1)=O
mmff94 0.0
gasteiger 1.9017024353112204
eem2015bm 0.0
SMILES: OC([C@H]1C[C@H]2C(F)(F)[C@H]2C1)=O
mmff94 0.0
gasteiger 3.727726652967509
eem2015bm 0.0
SMILES: OC([C@@H]1C[C@H]2C(F)(F)[C@H]2C1)=O
mmff94 0.0
gasteiger 1.7974225013047693
eem2015bm 0.0
SMILES: O=C(O)C1CCC1
mmff94 0.0
gasteiger 2.0494872133313216
eem2015bm 0.0
SMILES: O=C(O)C1(CF)CCC1
mmff94 0.0
gasteiger 0.5527204820280059
eem2015bm 0.0
SMILES: O=C(O)C1(C(F)F)CCC1
mmff94 0.0
gasteiger 4.327872648408048
eem2015bm 0.0
SMILES: O=C(O)C1(C(F)(F)F)CCC1
mmff94 0.0
gasteiger 3.3140571885138708
eem2015bm 0.0
SMILES: FC(F)(F)[C@@H]1C[C@@H](C(O)=O)C1
mmff94 0.0
gasteiger 1.846402727917698
eem2015bm 0.0
SMILES: FC(F)(F)[C@H]1C[C@@H](C(O)=O)C1
mmff94 0.

KeyboardInterrupt: 

In [26]:
import sys
import os
import math
import pandas as pd
from rdkit import Chem
from openbabel import openbabel
from openbabel import pybel

from rdkit import Chem
from rdkit.Chem import rdForceFieldHelpers
from rdkit.Chem import AllChem, Descriptors
import py3Dmol
import numpy as np

def find_conf_with_min_energy(mol):
    optimization_result = rdForceFieldHelpers.MMFFOptimizeMoleculeConfs(mol)
    
    min_energy, min_energy_conf_index = pow(10,5), None
    for index, (status, energy) in enumerate(optimization_result):
        if energy < min_energy and status == 0:
            min_energy_conf_index = index
            min_energy = min(min_energy, energy)

    return min_energy_conf_index, min_energy, mol

def rdkit_to_openbabel(rdkit_mol):
    sdf_writer = Chem.SDWriter('temp.sdf')
    sdf_writer.write(rdkit_mol)
    sdf_writer.close()

    ob_mol = next(pybel.readfile('sdf', 'temp.sdf'))

    return ob_mol.OBMol

    
smiles_string = 'O'
mol = Chem.MolFromSmiles(smiles_string)

mol = Chem.AddHs(mol)
rdForceFieldHelpers.MMFFSanitizeMolecule(mol)

num_rotatable_bonds = Descriptors.NumRotatableBonds(mol)
amount_of_confs = pow(3, num_rotatable_bonds + 3)
AllChem.EmbedMultipleConfs(mol, numConfs=amount_of_confs, randomSeed=3407)

min_energy_conf_index, min_energy, mol = find_conf_with_min_energy(mol)

mol = rdkit_to_openbabel(mol)

# You can get a list of charge models
#  using `obabel -L charges`
methods = [ 'mmff94', 'gasteiger', 'eem2015bm' ]

print(f"SMILES: {smiles_string}")
for method in methods:

    cm = openbabel.OBChargeModel.FindType(method)
    cm.ComputeCharges(mol)
    # but in C++ or Python we can get the dipole moment vector
    dipole = cm.GetDipoleMoment(mol)
    moment = math.sqrt(dipole.GetX()**2+dipole.GetY()**2+dipole.GetZ()**2)

    print(method, moment)

SMILES: O
mmff94 0.0
gasteiger 0.9411424317145922
eem2015bm 0.0


SMILES TO CASRN

In [1]:
import chemicals
import requests

def casrn_to_smiles(casrn):
    url = f"http://cactus.nci.nih.gov/chemical/structure/{casrn}/smiles"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err}")
    except Exception as err:
        print(f"An error occurred: {err}")

casrn_to_dipole_moment_dict = {}
casrn_to_smiles_dict = {}
smiles_to_dipole_moment = {}

int_index = 0
for int_index in range(50000000):
    CAS = chemicals.int_to_CAS(int_index)
    int_index += 1

    is_cas_valid = chemicals.identifiers.check_CAS(CAS)
    if is_cas_valid == False:
        continue
    
    CAS_dipole_moment = chemicals.dipole.dipole_moment(CAS)
    if CAS_dipole_moment == None:
        continue

    SMILES = casrn_to_smiles(CAS)
    casrn_to_dipole_moment_dict[CAS] = CAS_dipole_moment
    casrn_to_smiles_dict[CAS] = casrn_to_smiles_dict
    smiles_to_dipole_moment[SMILES] = CAS_dipole_moment
    
    print(f"CASRN: {CAS}, SMILES: {SMILES}, dipole_moment: {CAS_dipole_moment}")

CASRN: 50-00-0, SMILES: C=O, dipole_moment: 2.33
CASRN: 56-23-5, SMILES: ClC(Cl)(Cl)Cl, dipole_moment: 0.0
CASRN: 56-40-6, SMILES: NCC(O)=O, dipole_moment: 1.09
CASRN: 56-81-5, SMILES: OCC(O)CO, dipole_moment: 4.21
CASRN: 57-13-6, SMILES: NC(N)=O, dipole_moment: 3.83
CASRN: 57-55-6, SMILES: CC(O)CO, dipole_moment: 3.62988811742274
CASRN: 57-57-8, SMILES: O=C1CCO1, dipole_moment: 4.18
CASRN: 60-12-8, SMILES: OCCc1ccccc1, dipole_moment: 1.65
CASRN: 60-29-7, SMILES: CCOCC, dipole_moment: 1.15
CASRN: 60-34-4, SMILES: CNN, dipole_moment: 1.6294
CASRN: 60-35-5, SMILES: CC(N)=O, dipole_moment: 3.76
CASRN: 62-53-3, SMILES: Nc1ccccc1, dipole_moment: 1.53
CASRN: 64-17-5, SMILES: CCO, dipole_moment: 1.44
CASRN: 64-18-6, SMILES: OC=O, dipole_moment: 1.41
CASRN: 64-19-7, SMILES: CC(O)=O, dipole_moment: 1.7
CASRN: 65-85-0, SMILES: OC(=O)c1ccccc1, dipole_moment: 1.0
CASRN: 67-56-1, SMILES: CO, dipole_moment: 1.7
CASRN: 67-62-9, SMILES: [Cl-].CO[NH3+], dipole_moment: 2.7183
CASRN: 67-63-0, SMILES: CC(

KeyboardInterrupt: 

In [1]:
import pickle

a = smiles_to_dipole_moment

with open('smiles_to_dipole_moment.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('smiles_to_dipole_moment.pickle', 'rb') as handle:
    b = pickle.load(handle)

print(a == b)

NameError: name 'smiles_to_dipole_moment' is not defined

In [2]:
import pickle 
with open('smiles_to_dipole_moment.pickle', 'rb') as handle:
    smiles_to_dipole_moment = pickle.load(handle)

INVESTIGATE CORRELATION

In [3]:
smiles_to_dipole_moment_openbabel = {}
smiles_to_dipole_moment_centroid = {}

In [4]:
import sys
import os
import math
import pandas as pd
from rdkit import Chem
from openbabel import openbabel
from openbabel import pybel

from rdkit import Chem
from rdkit.Chem import rdForceFieldHelpers
from rdkit.Chem import AllChem, Descriptors
import py3Dmol
import numpy as np

from tqdm import tqdm

sys.path.insert(0, r'C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part\molecule_features')
from utils_3d import Molecule3DFeatures

def find_conf_with_min_energy(mol):
    optimization_result = rdForceFieldHelpers.MMFFOptimizeMoleculeConfs(mol)
    
    min_energy, min_energy_conf_index = pow(10,5), None
    for index, (status, energy) in enumerate(optimization_result):
        if energy < min_energy and status == 0:
            min_energy_conf_index = index
            min_energy = min(min_energy, energy)

    return min_energy_conf_index, min_energy, mol

def rdkit_to_openbabel(rdkit_mol):
    sdf_writer = Chem.SDWriter('temp.sdf')
    sdf_writer.write(rdkit_mol)
    sdf_writer.close()

    ob_mol = next(pybel.readfile('sdf', 'temp.sdf'))

    return ob_mol.OBMol


SMILES_list = smiles_to_dipole_moment.keys()

smiles_index = 0
for smiles_string in tqdm(SMILES_list):
    try:
        smiles_index += 1
        # if smiles_index > 200:
        #     break

        molFeatures = Molecule3DFeatures(smiles=smiles_string)
        dipole_moment_my_own = molFeatures.dipole_moment

        if smiles_string == None:
            continue
        
        mol = Chem.MolFromSmiles(smiles_string)

        # mol = Chem.MolFromSmiles('FC(F)(C1(NC2)CC2C1)F')
        mol = Chem.AddHs(mol)
        rdForceFieldHelpers.MMFFSanitizeMolecule(mol)

        num_rotatable_bonds = Descriptors.NumRotatableBonds(mol)
        amount_of_confs = 100
        AllChem.EmbedMultipleConfs(mol, numConfs=amount_of_confs, randomSeed=3407)
        # for atom in mol.GetAtoms():
        #     atom.SetAtomMapNum(atom.GetIdx())

        ri = mol.GetRingInfo()

        # min_energy_conf_index, min_energy, mol = find_conf_with_min_energy(mol)

        mol = rdkit_to_openbabel(mol)

        cm = openbabel.OBChargeModel.FindType('gasteiger')
        cm.ComputeCharges(mol)
        # but in C++ or Python we can get the dipole moment vector
        dipole = cm.GetDipoleMoment(mol)
        moment = math.sqrt(dipole.GetX()**2+dipole.GetY()**2+dipole.GetZ()**2)

        print(f"SMILES: {smiles_string}, dipole_moment openbabel: {moment}, dipole mine: {dipole_moment_my_own} dipole true: {smiles_to_dipole_moment[smiles_string]}")
        smiles_to_dipole_moment_openbabel[smiles_string] = moment
        smiles_to_dipole_moment_centroid[smiles_string] = dipole_moment_my_own
    except:
        print("Problem with SMILES")
        continue

  0%|          | 0/826 [00:00<?, ?it/s]

  0%|          | 3/826 [00:00<00:40, 20.20it/s]

SMILES: C=O, dipole_moment openbabel: 2.5437986891192055, dipole mine: 0.483347064917513 dipole true: 2.33
SMILES: ClC(Cl)(Cl)Cl, dipole_moment openbabel: 0.014668129366561656, dipole mine: 0.0044505148743712464 dipole true: 0.0
SMILES: NCC(O)=O, dipole_moment openbabel: 1.6466119535673613, dipole mine: 0.22285401641475816 dipole true: 1.09


  1%|          | 6/826 [00:00<01:11, 11.40it/s]

SMILES: OCC(O)CO, dipole_moment openbabel: 1.5332262776613115, dipole mine: 0.37216652018601953 dipole true: 4.21
SMILES: NC(N)=O, dipole_moment openbabel: 2.552923973975012, dipole mine: 0.4382508656364119 dipole true: 3.83
SMILES: CC(O)CO, dipole_moment openbabel: 2.399057458522901, dipole mine: 0.5395022078779912 dipole true: 3.62988811742274
SMILES: O=C1CCO1, dipole_moment openbabel: 4.604888759939361, dipole mine: 0.8593130103478042 dipole true: 4.18


...

SMILES: [3H][H], dipole_moment openbabel: 0.0, dipole mine: 0.0 dipole true: 0.0
SMILES: [O-][O-], dipole_moment openbabel: 0.0, dipole mine: 0.0 dipole true: 0.0
SMILES: [GeH3]O[GeH3], dipole_moment openbabel: 0.0, dipole mine: nan dipole true: 1.6282
SMILES: [SiH3][Se][SiH3], dipole_moment openbabel: 0.0, dipole mine: nan dipole true: 1.5206
SMILES: O[2H], dipole_moment openbabel: 0.8670489130162496, dipole mine: 0.2262426692388731 dipole true: 1.85
SMILES: O([3H])[3H], dipole_moment openbabel: 0.8670489130162496, dipole mine: 0.2262426692388731 dipole true: 2.1792
SMILES: FPF, dipole_moment openbabel: 2.6533990466026136, dipole mine: 0.5113848421020177 dipole true: 2.2873
SMILES: [O]Cl, dipole_moment openbabel: 0.7660859601911151, dipole mine: 0.1722157744322622 dipole true: 1.24
SMILES: NN=N, dipole_moment openbabel: 2.2082386553583686, dipole mine: 0.4128643216828411 dipole true: 1.4085
Problem with SMILES
SMILES: [Fe++], dipole_moment openbabel: 0.0, dipole mine: 0.0 dipole true:

[18:45:40] UFFTYPER: Unrecognized atom type: Fe2+2 (0)
[18:45:40] UFFTYPER: Unrecognized atom type: Fe2+2 (0)
[18:45:40] UFFTYPER: Unrecognized atom type: Ti+4 (0)
[18:45:40] UFFTYPER: Unrecognized atom type: Ti+4 (0)
[18:45:40] UFFTYPER: Unrecognized atom type: Ti+4 (0)
[18:45:40] UFFTYPER: Unrecognized atom type: Ti+4 (0)
[18:45:40] UFFTYPER: Unrecognized atom type: Cr1+3 (0)
[18:45:40] UFFTYPER: Unrecognized atom type: Cr1+3 (0)
 99%|█████████▉| 817/826 [05:35<00:00, 55.31it/s]

SMILES: NF, dipole_moment openbabel: 1.927087835658791, dipole mine: 0.3469283029802215 dipole true: 2.4507
SMILES: [Ti++], dipole_moment openbabel: 0.0, dipole mine: 0.0 dipole true: 0.0
SMILES: [Ti+4], dipole_moment openbabel: 0.0, dipole mine: 0.0 dipole true: 0.0
SMILES: [Cr+3], dipole_moment openbabel: 0.0, dipole mine: 0.0 dipole true: 0.0
SMILES: C\C=C/Cl, dipole_moment openbabel: 1.3725159297786231, dipole mine: 0.23764230979409806 dipole true: 1.67
SMILES: C/C=C/Cl, dipole_moment openbabel: 1.5827813994382096, dipole mine: 0.2672438320449034 dipole true: 1.97
SMILES: N(=O)[2H], dipole_moment openbabel: 2.1428982028165637, dipole mine: 0.37864673744101573 dipole true: 2.197
SMILES: [SiH3]S[SiH3], dipole_moment openbabel: 0.0, dipole mine: 0.3305926870273254 dipole true: 1.7852


[18:45:40] UFFTYPER: Warning: hybridization set to SP3 for atom 0
[18:45:40] UFFTYPER: Unrecognized charge state for atom: 0
[18:45:40] UFFTYPER: Warning: hybridization set to SP3 for atom 0
[18:45:40] UFFTYPER: Unrecognized charge state for atom: 0
[18:45:40] UFFTYPER: Unrecognized charge state for atom: 0
[18:45:40] UFFTYPER: Unrecognized charge state for atom: 1
[18:45:40] UFFTYPER: Unrecognized charge state for atom: 0
[18:45:40] UFFTYPER: Unrecognized charge state for atom: 1
[18:45:40] UFFTYPER: Unrecognized atom type: Ni5+2 (0)
[18:45:40] UFFTYPER: Unrecognized atom type: Ni5+2 (0)
100%|██████████| 826/826 [05:35<00:00,  2.46it/s]

SMILES: [AlH++].[Cl-].[Cl-], dipole_moment openbabel: 0.7376918034121134, dipole mine: nan dipole true: 2.1049
SMILES: CS[SiH3], dipole_moment openbabel: 0.0, dipole mine: 0.30749475701218765 dipole true: 1.8641
SMILES: [S-][S-], dipole_moment openbabel: 0.0, dipole mine: 0.0 dipole true: 0.0
SMILES: S.[Ni], dipole_moment openbabel: 0.0, dipole mine: 0.15570797302255462 dipole true: 8.5058
SMILES: [O--], dipole_moment openbabel: 0.0, dipole mine: 0.0 dipole true: 0.0
SMILES: [2H][H], dipole_moment openbabel: 0.0, dipole mine: 0.0 dipole true: 0.0


In [16]:
import pandas as pd

dict_for_df = {"SMILES": [],
               "true_dipole": [],
               "dipole_openbabel": [],
               "dipole_centroid": []}
for smiles, dipole in smiles_to_dipole_moment_centroid.items():
    if pd.isnull(smiles_to_dipole_moment[smiles]) or pd.isnull(smiles_to_dipole_moment_openbabel[smiles]) or pd.isnull(smiles_to_dipole_moment_centroid[smiles]):
        continue

    if 'f' not in smiles.lower() and 'n' not in smiles.lower() and 'c' not in smiles.lower() :
        continue

    # if 'f' not in smiles.lower() or smiles not in SMILES:
    #     continue

    dict_for_df["SMILES"].append(smiles)
    dict_for_df["true_dipole"].append(smiles_to_dipole_moment[smiles])
    dict_for_df["dipole_openbabel"].append(smiles_to_dipole_moment_openbabel[smiles])
    dict_for_df["dipole_centroid"].append(smiles_to_dipole_moment_centroid[smiles])

df = pd.DataFrame(dict_for_df)

In [7]:
SMILES = ['FC=C', 'FCF', 'CC(F)F', 'FC(F)=C', 'FC(Cl)Cl', 'FC(F)Cl',
       'FC(F)F', 'FC(F)(Br)Br', 'FC(F)(F)Br', 'CC(F)(F)Cl',
       'FC(Cl)(Cl)Cl', 'FC(F)(Cl)Cl', 'FC(F)(F)Cl', 'FC(F)(F)F',
       'OC(=O)C(F)(F)F', 'FC(F)(Cl)C(F)(F)Cl', 'FC(F)(F)C(F)(F)Cl',
       'FC(F)(F)C(F)(F)F', 'FC(F)(F)C(F)(F)C(F)(F)F',
       'FC1(F)C(F)(F)C(F)(F)C1(F)F', 'FC(F)=C(F)F',
       'FC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F',
       'FC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F',
       'FC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F', 'CCF',
       'FC(F)=O', 'NCC(F)(F)F', 'FC(F)C(F)(F)F',
       'FC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F',
       'FC1(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C1(F)F', 'FC=C(F)F',
       'FC(F)C(F)F', 'Fc1cccc(F)c1', 'FCBr', 'Fc1c(F)c(F)c(F)c(F)c1F',
       'CC(F)(F)F', 'Fc1ccccc1', 'CC(F)=O', 'CF', 'FCCl', 'FCCF',
       'FC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F', '[Fe++].[S-][S-]', 'FC=O',
       'FC#N', 'FC(F)Br', 'F\\C=C/F', 'F\\C=C\\F', 'FC(F)(F)I',
       'F[S](F)(=O)=O', 'FC#C', '[F-].[Na+]', 'FF', 'FOF', 'FOOF',
       'FN(F)F', 'FP(F)F', 'F[S](F)(F)F', 'F[Si](F)(F)F', '[F-].[K+]',
       '[Li+].[F-]', 'FN=O', 'FCl', '[O-][N+](F)=O', 'FN(F)N(F)F', 'FNF',
       'C.FF', 'OF', 'F[P](F)(F)=O', '[F-].[Cu+]', 'FSSF', 'FN=NF', 'FSF',
       'F[SiH2]F', 'FBr', '[F]', 'FPF', '[Fe++]', 'NF']

In [29]:
EXCEL_PATH = r'C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part\molecule_features\pKa_Predicition_Starting data_2024.01.19.xlsx'

df = pd.read_excel(EXCEL_PATH, sheet_name="Main_List")

for index, row in df.iterrows():
    if row['Smiles'] in smiles_to_dipole_moment.keys():
        print(f"SMILES: {row['Smiles']}, dipole_moment: {smiles_to_dipole_moment[row['Smiles']]}, dipole babel: {smiles_to_dipole_moment_openbabel[row['Smiles']]}, dipole own: {smiles_to_dipole_moment_centroid[row['Smiles']]}")

SMILES: NC1CCCCC1, dipole_moment: 1.31, dipole babel: 0.7053995311909147, dipole own: 0.1691891933702287
SMILES: C1CCNCC1, dipole_moment: 1.19, dipole babel: 0.7003072764006886, dipole own: 0.1487076303899343
SMILES: NCC(F)(F)F, dipole_moment: 1.33, dipole babel: 2.222173076588417, dipole own: 0.3408077660848498
SMILES: NC1CC1, dipole_moment: 1.19, dipole babel: 0.8373728389051304, dipole own: 0.19192143368007936
SMILES: CCCCN, dipole_moment: 1.39, dipole babel: 0.6902250415943879, dipole own: 0.1631436244622368


In [17]:
from scipy.stats import shapiro

print(shapiro(dict_for_df["true_dipole"]))
print(shapiro(dict_for_df["dipole_openbabel"]))
print(shapiro(dict_for_df["dipole_centroid"]))

df_for_corr = df[['true_dipole', 'dipole_openbabel', 'dipole_centroid']]
df_for_corr.corr(method='spearman')

ShapiroResult(statistic=0.8398455381393433, pvalue=2.6285212675323107e-25)
ShapiroResult(statistic=0.8983024954795837, pvalue=1.448439943048521e-20)
ShapiroResult(statistic=0.8415530323982239, pvalue=3.4488214845738073e-25)


,true_dipole,dipole_openbabel,dipole_centroid
true_dipole,1.000000,0.636014,0.644619
dipole_openbabel,0.636014,1.000000,0.887338
dipole_centroid,0.644619,0.887338,1.000000


In [ ]:
CASRN: 50-00-0, SMILES: C=O, dipole_moment: 2.33
CASRN: 56-23-5, SMILES: ClC(Cl)(Cl)Cl, dipole_moment: 0.0
CASRN: 56-40-6, SMILES: NCC(O)=O, dipole_moment: 1.09
CASRN: 56-81-5, SMILES: OCC(O)CO, dipole_moment: 4.21
CASRN: 57-13-6, SMILES: NC(N)=O, dipole_moment: 3.83
CASRN: 57-55-6, SMILES: CC(O)CO, dipole_moment: 3.62988811742274
CASRN: 57-57-8, SMILES: O=C1CCO1, dipole_moment: 4.18
CASRN: 60-12-8, SMILES: OCCc1ccccc1, dipole_moment: 1.65
CASRN: 60-29-7, SMILES: CCOCC, dipole_moment: 1.15
CASRN: 60-34-4, SMILES: CNN, dipole_moment: 1.6294
CASRN: 60-35-5, SMILES: CC(N)=O, dipole_moment: 3.76
CASRN: 62-53-3, SMILES: Nc1ccccc1, dipole_moment: 1.53
CASRN: 64-17-5, SMILES: CCO, dipole_moment: 1.44
CASRN: 64-18-6, SMILES: OC=O, dipole_moment: 1.41
CASRN: 64-19-7, SMILES: CC(O)=O, dipole_moment: 1.7
CASRN: 65-85-0, SMILES: OC(=O)c1ccccc1, dipole_moment: 1.0
CASRN: 67-56-1, SMILES: CO, dipole_moment: 1.7
CASRN: 67-62-9, SMILES: [Cl-].CO[NH3+], dipole_moment: 2.7183
CASRN: 67-63-0, SMILES: CC(C)O, dipole_moment: 1.58
CASRN: 67-64-1, SMILES: CC(C)=O, dipole_moment: 2.88
CASRN: 67-66-3, SMILES: ClC(Cl)Cl, dipole_moment: 1.01
CASRN: 67-68-5, SMILES: C[S](C)=O, dipole_moment: 3.96
CASRN: 67-71-0, SMILES: C[S](C)(=O)=O, dipole_moment: 4.49
CASRN: 67-72-1, SMILES: ClC(Cl)(Cl)C(Cl)(Cl)Cl, dipole_moment: 0.0
CASRN: 68-12-2, SMILES: CN(C)C=O, dipole_moment: 3.82
CASRN: 69-72-7, SMILES: OC(=O)c1ccccc1O, dipole_moment: 2.65
CASRN: 71-23-8, SMILES: CCCO, dipole_moment: 1.55
CASRN: 71-36-3, SMILES: CCCCO, dipole_moment: 1.66
CASRN: 71-41-0, SMILES: CCCCCO, dipole_moment: 1.70000359751052
CASRN: 71-43-2, SMILES: c1ccccc1, dipole_moment: 0.0
CASRN: 71-55-6, SMILES: CC(Cl)(Cl)Cl, dipole_moment: 1.78
CASRN: 74-82-8, SMILES: C, dipole_moment: 0.0
CASRN: 74-83-9, SMILES: CBr, dipole_moment: 1.81
CASRN: 74-84-0, SMILES: CC, dipole_moment: 0.0
CASRN: 74-85-1, SMILES: C=C, dipole_moment: 0.0
CASRN: 74-86-2, SMILES: C#C, dipole_moment: 0.0
CASRN: 74-87-3, SMILES: CCl, dipole_moment: 1.87
CASRN: 74-88-4, SMILES: CI, dipole_moment: 1.62
CASRN: 74-89-5, SMILES: CN, dipole_moment: 1.31
CASRN: 74-90-8, SMILES: C#N, dipole_moment: 2.98
CASRN: 74-93-1, SMILES: CS, dipole_moment: 1.52
CASRN: 74-95-3, SMILES: BrCBr, dipole_moment: 1.43
CASRN: 74-96-4, SMILES: CCBr, dipole_moment: 2.03
CASRN: 74-97-5, SMILES: ClCBr, dipole_moment: 2.0001
CASRN: 74-98-6, SMILES: CCC, dipole_moment: 0.08
CASRN: 74-99-7, SMILES: CC#C, dipole_moment: 0.78
CASRN: 75-00-3, SMILES: CCCl, dipole_moment: 2.05
CASRN: 75-01-4, SMILES: ClC=C, dipole_moment: 1.45
CASRN: 75-02-5, SMILES: FC=C, dipole_moment: 1.43
CASRN: 75-04-7, SMILES: CCN, dipole_moment: 1.22
CASRN: 75-05-8, SMILES: CC#N, dipole_moment: 3.92
CASRN: 75-07-0, SMILES: CC=O, dipole_moment: 2.69
CASRN: 75-08-1, SMILES: CCS, dipole_moment: 1.52
CASRN: 75-09-2, SMILES: ClCCl, dipole_moment: 1.6
CASRN: 75-10-5, SMILES: FCF, dipole_moment: 1.97
CASRN: 75-12-7, SMILES: NC=O, dipole_moment: 3.73
CASRN: 75-15-0, SMILES: S=C=S, dipole_moment: 0.0
CASRN: 75-18-3, SMILES: CSC, dipole_moment: 1.5
CASRN: 75-19-4, SMILES: C1CC1, dipole_moment: 0.0
CASRN: 75-21-8, SMILES: C1CO1, dipole_moment: 1.89
CASRN: 75-25-2, SMILES: BrC(Br)Br, dipole_moment: 0.99
CASRN: 75-26-3, SMILES: CC(C)Br, dipole_moment: 2.21
CASRN: 75-28-5, SMILES: CC(C)C, dipole_moment: 0.13
CASRN: 75-29-6, SMILES: CC(C)Cl, dipole_moment: 2.17
CASRN: 75-31-0, SMILES: CC(C)N, dipole_moment: 1.19
CASRN: 75-34-3, SMILES: CC(Cl)Cl, dipole_moment: 2.06
CASRN: 75-35-4, SMILES: ClC(Cl)=C, dipole_moment: 1.34
CASRN: 75-36-5, SMILES: CC(Cl)=O, dipole_moment: 2.72
CASRN: 75-37-6, SMILES: CC(F)F, dipole_moment: 2.27
CASRN: 75-38-7, SMILES: FC(F)=C, dipole_moment: 1.38
CASRN: 75-43-4, SMILES: FC(Cl)Cl, dipole_moment: 1.29
CASRN: 75-44-5, SMILES: ClC(Cl)=O, dipole_moment: 1.17
CASRN: 75-45-6, SMILES: FC(F)Cl, dipole_moment: 1.4
CASRN: 75-46-7, SMILES: FC(F)F, dipole_moment: 1.65
CASRN: 75-50-3, SMILES: CN(C)C, dipole_moment: 0.61
CASRN: 75-52-5, SMILES: C[N+]([O-])=O, dipole_moment: 3.46
CASRN: 75-56-9, SMILES: CC1CO1, dipole_moment: 2.01
CASRN: 75-61-6, SMILES: FC(F)(Br)Br, dipole_moment: 0.66
CASRN: 75-63-8, SMILES: FC(F)(F)Br, dipole_moment: 0.64
CASRN: 75-65-0, SMILES: CC(C)(C)O, dipole_moment: 1.74
CASRN: 75-68-3, SMILES: CC(F)(F)Cl, dipole_moment: 2.14
CASRN: 75-69-4, SMILES: FC(Cl)(Cl)Cl, dipole_moment: 0.45
CASRN: 75-71-8, SMILES: FC(F)(Cl)Cl, dipole_moment: 0.51
CASRN: 75-72-9, SMILES: FC(F)(F)Cl, dipole_moment: 0.5
CASRN: 75-73-0, SMILES: FC(F)(F)F, dipole_moment: 0.0
CASRN: 75-76-3, SMILES: C[Si](C)(C)C, dipole_moment: 0.0
CASRN: 75-83-2, SMILES: CCC(C)(C)C, dipole_moment: 0.0
CASRN: 75-85-4, SMILES: CCC(C)(C)O, dipole_moment: 1.70000359751052
CASRN: 76-01-7, SMILES: ClC(Cl)C(Cl)(Cl)Cl, dipole_moment: 0.92
CASRN: 76-05-1, SMILES: OC(=O)C(F)(F)F, dipole_moment: 2.28
CASRN: 76-14-2, SMILES: FC(F)(Cl)C(F)(F)Cl, dipole_moment: 0.56
CASRN: 76-15-3, SMILES: FC(F)(F)C(F)(F)Cl, dipole_moment: 0.52
CASRN: 76-16-4, SMILES: FC(F)(F)C(F)(F)F, dipole_moment: 0.0
CASRN: 76-19-7, SMILES: FC(F)(F)C(F)(F)C(F)(F)F, dipole_moment: 0.0
CASRN: 78-75-1, SMILES: CC(Br)CBr, dipole_moment: 1.24
CASRN: 78-76-2, SMILES: CCC(C)Br, dipole_moment: 2.23
CASRN: 78-78-4, SMILES: CCC(C)C, dipole_moment: 0.13
CASRN: 78-79-5, SMILES: CC(=C)C=C, dipole_moment: 0.25
CASRN: 78-80-8, SMILES: CC(=C)C#C, dipole_moment: 0.51
CASRN: 78-81-9, SMILES: CC(C)CN, dipole_moment: 1.27
CASRN: 78-83-1, SMILES: CC(C)CO, dipole_moment: 1.64
CASRN: 78-85-3, SMILES: CC(=C)C=O, dipole_moment: 2.68
CASRN: 78-86-4, SMILES: CCC(C)Cl, dipole_moment: 2.04
CASRN: 78-87-5, SMILES: CC(Cl)CCl, dipole_moment: 1.17
CASRN: 78-92-2, SMILES: CCC(C)O, dipole_moment: 1.7
CASRN: 78-93-3, SMILES: CCC(C)=O, dipole_moment: 2.78
CASRN: 78-99-9, SMILES: CCC(Cl)Cl, dipole_moment: 2.081
CASRN: 79-00-5, SMILES: ClCC(Cl)Cl, dipole_moment: 1.5
CASRN: 79-01-6, SMILES: ClC=C(Cl)Cl, dipole_moment: 0.77
CASRN: 79-09-4, SMILES: CCC(O)=O, dipole_moment: 1.751
CASRN: 79-10-7, SMILES: OC(=O)C=C, dipole_moment: 1.4599896871365
CASRN: 79-11-8, SMILES: OC(=O)CCl, dipole_moment: 2.31
CASRN: 79-20-9, SMILES: COC(C)=O, dipole_moment: 1.72
CASRN: 79-24-3, SMILES: CC[N+]([O-])=O, dipole_moment: 3.65
CASRN: 79-27-6, SMILES: BrC(Br)C(Br)Br, dipole_moment: 1.3
CASRN: 79-29-8, SMILES: CC(C)C(C)C, dipole_moment: 0.2
CASRN: 79-31-2, SMILES: CC(C)C(O)=O, dipole_moment: 1.09
CASRN: 79-34-5, SMILES: ClC(Cl)C(Cl)Cl, dipole_moment: 1.29
CASRN: 79-41-4, SMILES: CC(=C)C(O)=O, dipole_moment: 1.64999820124474
CASRN: 79-46-9, SMILES: CC(C)[N+]([O-])=O, dipole_moment: 3.73
CASRN: 80-62-6, SMILES: COC(=O)C(C)=C, dipole_moment: 1.96999676224053
CASRN: 83-32-9, SMILES: C1Cc2cccc3cccc1c23, dipole_moment: 0.254823662025878
CASRN: 84-66-2, SMILES: CCOC(=O)c1ccccc1C(=O)OCC, dipole_moment: 2.90001319087192
CASRN: 85-01-8, SMILES: c1ccc2c(c1)ccc3ccccc23, dipole_moment: 0.0
CASRN: 88-74-4, SMILES: Nc1ccccc1[N+]([O-])=O, dipole_moment: 4.06
CASRN: 88-99-3, SMILES: OC(=O)c1ccccc1C(O)=O, dipole_moment: 2.6
CASRN: 90-12-0, SMILES: Cc1cccc2ccccc12, dipole_moment: 0.5
CASRN: 91-20-3, SMILES: c1ccc2ccccc2c1, dipole_moment: 0.0
CASRN: 91-22-5, SMILES: c1ccc2ncccc2c1, dipole_moment: 2.3
CASRN: 91-57-6, SMILES: Cc1ccc2ccccc2c1, dipole_moment: 0.4
CASRN: 92-52-4, SMILES: c1ccc(cc1)c2ccccc2, dipole_moment: 0.0
CASRN: 92-67-1, SMILES: Nc1ccc(cc1)c2ccccc2, dipole_moment: 1.76
CASRN: 92-94-4, SMILES: c1ccc(cc1)c2ccc(cc2)c3ccccc3, dipole_moment: 0.7
CASRN: 93-58-3, SMILES: COC(=O)c1ccccc1, dipole_moment: 2.53000923361034
CASRN: 95-47-6, SMILES: Cc1ccccc1C, dipole_moment: 0.629564341475699
CASRN: 95-48-7, SMILES: Cc1ccccc1O, dipole_moment: 1.45099591082971
CASRN: 95-65-8, SMILES: Cc1ccc(O)cc1C, dipole_moment: 1.7
CASRN: 95-87-4, SMILES: Cc1ccc(C)c(O)c1, dipole_moment: 1.5
CASRN: 95-93-2, SMILES: Cc1cc(C)c(C)cc1C, dipole_moment: 0.0
CASRN: 96-22-0, SMILES: CCC(=O)CC, dipole_moment: 2.7
CASRN: 96-37-7, SMILES: CC1CCCC1, dipole_moment: 0.0
CASRN: 96-48-0, SMILES: O=C1CCCO1, dipole_moment: 3.82
CASRN: 96-49-1, SMILES: O=C1OCCO1, dipole_moment: 4.51007902531448
CASRN: 97-62-1, SMILES: CCOC(=O)C(C)C, dipole_moment: 2.1
CASRN: 97-63-2, SMILES: CCOC(=O)C(C)=C, dipole_moment: 2.14999220539387
CASRN: 97-64-3, SMILES: CCOC(=O)C(C)O, dipole_moment: 2.39998920746843
CASRN: 97-99-4, SMILES: OCC1CCCO1, dipole_moment: 2.12001295103788
CASRN: 98-01-1, SMILES: O=Cc1occc1, dipole_moment: 3.54
CASRN: 98-82-8, SMILES: CC(C)c1ccccc1, dipole_moment: 0.8
CASRN: 98-86-2, SMILES: CC(=O)c1ccccc1, dipole_moment: 3.02
CASRN: 98-95-3, SMILES: [O-][N+](=O)c1ccccc1, dipole_moment: 4.22
CASRN: 99-08-1, SMILES: Cc1cccc(c1)[N+]([O-])=O, dipole_moment: 4.23
CASRN: 99-09-2, SMILES: Nc1cccc(c1)[N+]([O-])=O, dipole_moment: 4.9
CASRN: 99-65-0, SMILES: [O-][N+](=O)c1cccc(c1)[N+]([O-])=O, dipole_moment: 3.837
CASRN: 99-87-6, SMILES: CC(C)c1ccc(C)cc1, dipole_moment: 0.0
CASRN: 99-99-0, SMILES: Cc1ccc(cc1)[N+]([O-])=O, dipole_moment: 4.44
CASRN: 100-21-0, SMILES: OC(=O)c1ccc(cc1)C(O)=O, dipole_moment: 2.60219927809955
CASRN: 100-25-4, SMILES: [O-][N+](=O)c1ccc(cc1)[N+]([O-])=O, dipole_moment: 0.0
CASRN: 100-41-4, SMILES: CCc1ccccc1, dipole_moment: 0.59
CASRN: 100-46-9, SMILES: NCc1ccccc1, dipole_moment: 1.38
CASRN: 100-47-0, SMILES: N#Cc1ccccc1, dipole_moment: 4.18
CASRN: 100-51-6, SMILES: OCc1ccccc1, dipole_moment: 1.7
CASRN: 100-52-7, SMILES: O=Cc1ccccc1, dipole_moment: 3.14
CASRN: 100-66-3, SMILES: COc1ccccc1, dipole_moment: 1.26
CASRN: 101-81-5, SMILES: C(c1ccccc1)c2ccccc2, dipole_moment: 0.4
CASRN: 104-51-8, SMILES: CCCCc1ccccc1, dipole_moment: 0.4
CASRN: 104-76-7, SMILES: CCCCC(CC)CO, dipole_moment: 1.8
CASRN: 105-05-5, SMILES: CCc1ccc(CC)cc1, dipole_moment: 0.0
CASRN: 105-37-3, SMILES: CCOC(=O)CC, dipole_moment: 1.7519876245638
CASRN: 105-54-4, SMILES: CCCC(=O)OCC, dipole_moment: 1.8
CASRN: 105-58-8, SMILES: CCOC(=O)OCC, dipole_moment: 1.09999880082983
CASRN: 105-66-8, SMILES: CCCOC(=O)CCC, dipole_moment: 1.8
CASRN: 105-67-9, SMILES: Cc1ccc(O)c(C)c1, dipole_moment: 2.0
CASRN: 106-36-5, SMILES: CCCOC(=O)CC, dipole_moment: 1.79000131908719
CASRN: 106-42-3, SMILES: Cc1ccc(C)cc1, dipole_moment: 0.0
CASRN: 106-44-5, SMILES: Cc1ccc(O)cc1, dipole_moment: 1.55892122651125
CASRN: 106-49-0, SMILES: Cc1ccc(N)cc1, dipole_moment: 1.31
CASRN: 106-89-8, SMILES: ClCC1CO1, dipole_moment: 1.8
CASRN: 106-93-4, SMILES: BrCCBr, dipole_moment: 1.04
CASRN: 106-94-5, SMILES: CCCBr, dipole_moment: 2.18
CASRN: 106-97-8, SMILES: CCCC, dipole_moment: 0.0
CASRN: 106-98-9, SMILES: CCC=C, dipole_moment: 0.36
CASRN: 106-99-0, SMILES: C=CC=C, dipole_moment: 0.0
CASRN: 107-00-6, SMILES: CCC#C, dipole_moment: 0.78
CASRN: 107-02-8, SMILES: C=CC=O, dipole_moment: 3.12
CASRN: 107-05-1, SMILES: ClCC=C, dipole_moment: 1.94
CASRN: 107-06-2, SMILES: ClCCCl, dipole_moment: 2.94
CASRN: 107-07-3, SMILES: OCCCl, dipole_moment: 1.78
CASRN: 107-10-8, SMILES: CCCN, dipole_moment: 1.17
CASRN: 107-12-0, SMILES: CCC#N, dipole_moment: 4.02
CASRN: 107-13-1, SMILES: C=CC#N, dipole_moment: 3.87
CASRN: 107-15-3, SMILES: NCCN, dipole_moment: 1.99
CASRN: 107-18-6, SMILES: OCC=C, dipole_moment: 1.6
CASRN: 107-19-7, SMILES: OCC#C, dipole_moment: 1.81
CASRN: 107-20-0, SMILES: ClCC=O, dipole_moment: 2.72
CASRN: 107-21-1, SMILES: OCCO, dipole_moment: 2.3084025854109
CASRN: 107-29-9, SMILES: C/C=N/O, dipole_moment: 0.94
CASRN: 107-31-3, SMILES: COC=O, dipole_moment: 1.77
CASRN: 107-41-5, SMILES: CC(O)CC(C)(C)O, dipole_moment: 2.9
CASRN: 107-83-5, SMILES: CCCC(C)C, dipole_moment: 0.0
CASRN: 107-87-9, SMILES: CCCC(C)=O, dipole_moment: 2.71998776846422
CASRN: 107-92-6, SMILES: CCCC(O)=O, dipole_moment: 1.23
CASRN: 107-93-7, SMILES: C/C=C/C(O)=O, dipole_moment: 2.13
CASRN: 108-03-2, SMILES: CCC[N+]([O-])=O, dipole_moment: 3.66
CASRN: 108-08-7, SMILES: CC(C)CC(C)C, dipole_moment: 0.0
CASRN: 108-10-1, SMILES: CC(C)CC(C)=O, dipole_moment: 2.8
CASRN: 108-20-3, SMILES: CC(C)OC(C)C, dipole_moment: 1.13000803444017
CASRN: 108-21-4, SMILES: CC(C)OC(C)=O, dipole_moment: 1.75000899377631
CASRN: 108-24-7, SMILES: CC(=O)OC(C)=O, dipole_moment: 3.0
CASRN: 108-38-3, SMILES: Cc1cccc(C)c1, dipole_moment: 0.299792543559857
CASRN: 108-39-4, SMILES: Cc1cccc(O)c1, dipole_moment: 1.58890048086724
CASRN: 108-44-1, SMILES: Cc1cccc(N)c1, dipole_moment: 1.49
CASRN: 108-46-3, SMILES: Oc1cccc(O)c1, dipole_moment: 2.09
CASRN: 108-47-4, SMILES: Cc1ccnc(C)c1, dipole_moment: 2.3
CASRN: 108-48-5, SMILES: Cc1cccc(C)n1, dipole_moment: 1.7
CASRN: 108-64-5, SMILES: CCOC(=O)CC(C)C, dipole_moment: 1.96999676224053
CASRN: 108-67-8, SMILES: Cc1cc(C)cc(C)c1, dipole_moment: 0.1
CASRN: 108-68-9, SMILES: Cc1cc(C)cc(O)c1, dipole_moment: 1.8
CASRN: 108-86-1, SMILES: Brc1ccccc1, dipole_moment: 1.7
CASRN: 108-87-2, SMILES: CC1CCCCC1, dipole_moment: 0.0
CASRN: 108-88-3, SMILES: Cc1ccccc1, dipole_moment: 0.33
CASRN: 108-90-7, SMILES: Clc1ccccc1, dipole_moment: 1.69
CASRN: 108-91-8, SMILES: NC1CCCCC1, dipole_moment: 1.31
CASRN: 108-93-0, SMILES: OC1CCCCC1, dipole_moment: 1.86000287800842
CASRN: 108-94-1, SMILES: O=C1CCCCC1, dipole_moment: 3.02
CASRN: 108-95-2, SMILES: Oc1ccccc1, dipole_moment: 1.22
CASRN: 108-99-6, SMILES: Cc1cccnc1, dipole_moment: 2.4
CASRN: 109-06-8, SMILES: Cc1ccccn1, dipole_moment: 1.9
CASRN: 109-60-4, SMILES: CCCOC(C)=O, dipole_moment: 1.79000131908719
CASRN: 109-66-0, SMILES: CCCCC, dipole_moment: 0.0
CASRN: 109-67-1, SMILES: CCCC=C, dipole_moment: 0.39
CASRN: 109-69-3, SMILES: CCCCCl, dipole_moment: 2.05
CASRN: 109-73-9, SMILES: CCCCN, dipole_moment: 1.39
CASRN: 109-74-0, SMILES: CCCC#N, dipole_moment: 4.07
CASRN: 109-77-3, SMILES: N#CCC#N, dipole_moment: 3.73
CASRN: 109-83-1, SMILES: CNCCO, dipole_moment: 2.159
CASRN: 109-86-4, SMILES: COCCO, dipole_moment: 2.04
CASRN: 109-89-7, SMILES: CCNCC, dipole_moment: 0.92
CASRN: 109-92-2, SMILES: CCOC=C, dipole_moment: 1.27
CASRN: 109-93-3, SMILES: C=COC=C, dipole_moment: 0.78
CASRN: 109-94-4, SMILES: CCOC=O, dipole_moment: 1.98
CASRN: 109-97-7, SMILES: [nH]1cccc1, dipole_moment: 1.84
CASRN: 109-99-9, SMILES: C1CCOC1, dipole_moment: 1.63
CASRN: 110-00-9, SMILES: o1cccc1, dipole_moment: 0.66
CASRN: 110-02-1, SMILES: s1cccc1, dipole_moment: 0.55
CASRN: 110-15-6, SMILES: OC(=O)CCC(O)=O, dipole_moment: 2.2
CASRN: 110-17-8, SMILES: OC(=O)\C=C\C(O)=O, dipole_moment: 2.45
CASRN: 110-19-0, SMILES: CC(C)COC(C)=O, dipole_moment: 1.871
CASRN: 110-54-3, SMILES: CCCCCC, dipole_moment: 0.0
CASRN: 110-56-5, SMILES: ClCCCCCl, dipole_moment: 2.22
CASRN: 110-59-8, SMILES: CCCCC#N, dipole_moment: 4.12
CASRN: 110-62-3, SMILES: CCCCC=O, dipole_moment: 2.57000155892123
CASRN: 110-63-4, SMILES: OCCCCO, dipole_moment: 3.93
CASRN: 110-71-4, SMILES: COCCOC, dipole_moment: 1.71
CASRN: 110-74-7, SMILES: CCCOC=O, dipole_moment: 1.9
CASRN: 110-82-7, SMILES: C1CCCCC1, dipole_moment: 0.61
CASRN: 110-83-8, SMILES: C1CCC=CC1, dipole_moment: 0.33
CASRN: 110-85-0, SMILES: C1CNCCN1, dipole_moment: 1.47
CASRN: 110-86-1, SMILES: c1ccncc1, dipole_moment: 2.19
CASRN: 110-88-3, SMILES: C1OCOCO1, dipole_moment: 2.08
CASRN: 110-89-4, SMILES: C1CCNCC1, dipole_moment: 1.19
CASRN: 110-94-1, SMILES: OC(=O)CCCC(O)=O, dipole_moment: 2.64
CASRN: 111-15-9, SMILES: CCOCCOC(C)=O, dipole_moment: 2.25000299792544
CASRN: 111-27-3, SMILES: CCCCCCO, dipole_moment: 1.64999820124474
CASRN: 111-29-5, SMILES: OCCCCCO, dipole_moment: 2.37000995311245
CASRN: 111-34-2, SMILES: CCCCOC=C, dipole_moment: 1.25001498962718
CASRN: 111-42-2, SMILES: OCCNCCO, dipole_moment: 0.851
CASRN: 111-43-3, SMILES: CCCOCCC, dipole_moment: 1.21
CASRN: 111-46-6, SMILES: OCCOCCO, dipole_moment: 5.486
CASRN: 111-65-9, SMILES: CCCCCCCC, dipole_moment: 0.0
CASRN: 111-66-0, SMILES: CCCCCCC=C, dipole_moment: 0.3
CASRN: 111-70-6, SMILES: CCCCCCCO, dipole_moment: 1.7
CASRN: 111-76-2, SMILES: CCCCOCCO, dipole_moment: 2.07999064647264
CASRN: 111-84-2, SMILES: CCCCCCCCC, dipole_moment: 0.0
CASRN: 111-87-5, SMILES: CCCCCCCCO, dipole_moment: 1.64999820124474
CASRN: 111-92-2, SMILES: CCCCNCCCC, dipole_moment: 1.1
CASRN: 112-27-6, SMILES: OCCOCCOCCO, dipole_moment: 5.58
CASRN: 112-30-1, SMILES: CCCCCCCCCCO, dipole_moment: 1.8
CASRN: 112-40-3, SMILES: CCCCCCCCCCCC, dipole_moment: 0.0
CASRN: 112-80-1, SMILES: CCCCCCCC/C=C/CCCCCCCC(O)=O, dipole_moment: 1.44
CASRN: 112-92-5, SMILES: CCCCCCCCCCCCCCCCCCO, dipole_moment: 1.7
CASRN: 112-95-8, SMILES: CCCCCCCCCCCCCCCCCCCC, dipole_moment: 0.0
CASRN: 115-07-1, SMILES: CC=C, dipole_moment: 0.36
CASRN: 115-08-2, SMILES: NC=S, dipole_moment: 2.9905
CASRN: 115-10-6, SMILES: COC, dipole_moment: 1.3
CASRN: 115-11-7, SMILES: CC(C)=C, dipole_moment: 0.5
CASRN: 115-25-3, SMILES: FC1(F)C(F)(F)C(F)(F)C1(F)F, dipole_moment: 0.0
CASRN: 116-14-3, SMILES: FC(F)=C(F)F, dipole_moment: 0.0
CASRN: 117-84-0, SMILES: CCCCCCCCOC(=O)c1ccccc1C(=O)OCCCCCCCC, dipole_moment: 2.839
CASRN: 118-90-1, SMILES: Cc1ccccc1C(O)=O, dipole_moment: 1.7
CASRN: 119-36-8, SMILES: COC(=O)c1ccccc1O, dipole_moment: 2.46999076638966
CASRN: 119-65-3, SMILES: c1ccc2cnccc2c1, dipole_moment: 2.7
CASRN: 120-12-7, SMILES: c1ccc2cc3ccccc3cc2c1, dipole_moment: 0.0
CASRN: 120-80-9, SMILES: Oc1ccccc1O, dipole_moment: 2.599
CASRN: 120-92-3, SMILES: O=C1CCCC1, dipole_moment: 3.25
CASRN: 121-44-8, SMILES: CCN(CC)CC, dipole_moment: 0.9
CASRN: 121-91-5, SMILES: OC(=O)c1cccc(c1)C(O)=O, dipole_moment: 2.27
CASRN: 123-05-7, SMILES: CCCCC(CC)C=O, dipole_moment: 2.66
CASRN: 123-38-6, SMILES: CCC=O, dipole_moment: 2.52
CASRN: 123-42-2, SMILES: CC(=O)CC(C)(C)O, dipole_moment: 3.24015781079493
CASRN: 123-51-3, SMILES: CC(C)CCO, dipole_moment: 1.79998441078774
CASRN: 123-72-8, SMILES: CCCC=O, dipole_moment: 2.72
CASRN: 123-73-9, SMILES: C/C=C/C=O, dipole_moment: 3.67
CASRN: 123-86-4, SMILES: CCCCOC(C)=O, dipole_moment: 1.8401266323704
CASRN: 123-91-1, SMILES: C1COCCO1, dipole_moment: 0.0
CASRN: 123-92-2, SMILES: CC(C)CCOC(C)=O, dipole_moment: 1.8
CASRN: 123-96-6, SMILES: CCCCCCC(C)O, dipole_moment: 1.6
CASRN: 123-99-9, SMILES: OC(=O)CCCCCCCC(O)=O, dipole_moment: 2.35
CASRN: 124-04-9, SMILES: OC(=O)CCCCC(O)=O, dipole_moment: 2.32
CASRN: 124-18-5, SMILES: CCCCCCCCCC, dipole_moment: 0.0
CASRN: 124-38-9, SMILES: O=C=O, dipole_moment: 0.0
CASRN: 124-40-3, SMILES: CNC, dipole_moment: 1.03
CASRN: 127-18-4, SMILES: ClC(Cl)=C(Cl)Cl, dipole_moment: 0.0
CASRN: 131-11-3, SMILES: COC(=O)c1ccccc1C(=O)OC, dipole_moment: 2.8
CASRN: 141-43-5, SMILES: NCCO, dipole_moment: 0.777
CASRN: 141-46-8, SMILES: OCC=O, dipole_moment: 2.73
CASRN: 141-78-6, SMILES: CCOC(C)=O, dipole_moment: 1.78
CASRN: 141-93-5, SMILES: CCc1cccc(CC)c1, dipole_moment: 0.36
CASRN: 142-28-9, SMILES: ClCCCCl, dipole_moment: 2.081
CASRN: 142-29-0, SMILES: C1CC=CC1, dipole_moment: 0.19
CASRN: 142-68-7, SMILES: C1CCOCC1, dipole_moment: 1.74
CASRN: 142-82-5, SMILES: CCCCCCC, dipole_moment: 0.0
CASRN: 142-84-7, SMILES: CCCNCCC, dipole_moment: 1.07
CASRN: 143-08-8, SMILES: CCCCCCCCCO, dipole_moment: 1.7
CASRN: 151-51-9, SMILES: N=C=N, dipole_moment: 3.3696
CASRN: 151-56-4, SMILES: C1CN1, dipole_moment: 1.9
CASRN: 156-59-2, SMILES: Cl\C=C/Cl, dipole_moment: 1.9
CASRN: 156-60-5, SMILES: Cl\C=C\Cl, dipole_moment: 0.0
CASRN: 156-87-6, SMILES: NCCCO, dipole_moment: 2.69
CASRN: 157-16-4, SMILES: C1C=N1, dipole_moment: 2.4388
CASRN: 157-18-6, SMILES: O1C=C1, dipole_moment: 2.2342
CASRN: 157-22-2, SMILES: C1N=N1, dipole_moment: 1.59
CASRN: 157-26-6, SMILES: C1OO1, dipole_moment: 2.48
CASRN: 157-33-5, SMILES: C1C2CC12, dipole_moment: 0.68
CASRN: 185-94-4, SMILES: C1CC2CC12, dipole_moment: 0.26
CASRN: 275-51-4, SMILES: c1ccc2cccc2cc1, dipole_moment: 0.8
CASRN: 277-10-1, SMILES: C12C3C4C1C5C2C3C45, dipole_moment: 0.0
CASRN: 279-23-2, SMILES: C1CC2CCC1C2, dipole_moment: 0.09
CASRN: 285-58-5, SMILES: C1CC2CC2C1, dipole_moment: 0.19
CASRN: 287-23-0, SMILES: C1CCC1, dipole_moment: 0.0
CASRN: 287-27-4, SMILES: C1CSC1, dipole_moment: 1.85
CASRN: 287-92-3, SMILES: C1CCCC1, dipole_moment: 0.0
CASRN: 288-14-2, SMILES: o1cccn1, dipole_moment: 2.9
CASRN: 288-42-6, SMILES: o1ccnc1, dipole_moment: 1.5
CASRN: 290-37-9, SMILES: c1cnccn1, dipole_moment: 0.0
CASRN: 290-87-9, SMILES: c1ncncn1, dipole_moment: 0.0
CASRN: 291-64-5, SMILES: C1CCCCCC1, dipole_moment: 0.0
CASRN: 292-64-8, SMILES: C1CCCCCCC1, dipole_moment: 0.0
CASRN: 302-01-2, SMILES: NN, dipole_moment: 1.75
CASRN: 307-34-6, SMILES: FC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F, dipole_moment: 0.0
CASRN: 307-45-9, SMILES: FC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F, dipole_moment: 0.0
CASRN: 334-88-3, SMILES: C=[N+]=[N-], dipole_moment: 1.5
CASRN: 335-57-9, SMILES: FC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F, dipole_moment: 0.0
CASRN: 352-93-2, SMILES: CCSCC, dipole_moment: 1.56
CASRN: 353-36-6, SMILES: CCF, dipole_moment: 1.96
CASRN: 353-50-4, SMILES: FC(F)=O, dipole_moment: 0.95
CASRN: 353-85-5, SMILES: NCC(F)(F)F, dipole_moment: 1.33
CASRN: 354-33-6, SMILES: FC(F)C(F)(F)F, dipole_moment: 1.541
CASRN: 355-42-0, SMILES: FC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F, dipole_moment: 0.0
CASRN: 355-68-0, SMILES: FC1(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C1(F)F, dipole_moment: 0.0
CASRN: 359-11-5, SMILES: FC=C(F)F, dipole_moment: 1.4
CASRN: 359-35-3, SMILES: FC(F)C(F)F, dipole_moment: 0.0
CASRN: 372-18-9, SMILES: Fc1cccc(F)c1, dipole_moment: 1.58
CASRN: 373-52-4, SMILES: FCBr, dipole_moment: 2.1681
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/375-96-2/smiles
CASRN: 375-96-2, SMILES: None, dipole_moment: 0.0
CASRN: 392-56-3, SMILES: Fc1c(F)c(F)c(F)c(F)c1F, dipole_moment: 0.0
CASRN: 409-21-2, SMILES: [C-]#[Si+], dipole_moment: 1.1802
CASRN: 420-04-2, SMILES: NC#N, dipole_moment: 4.32
CASRN: 420-05-3, SMILES: OC#N, dipole_moment: 3.997
CASRN: 420-12-2, SMILES: C1CS1, dipole_moment: 1.85
CASRN: 420-46-2, SMILES: CC(F)(F)F, dipole_moment: 2.32
CASRN: 460-12-8, SMILES: C#CC#C, dipole_moment: 0.0
CASRN: 460-19-5, SMILES: NCCN, dipole_moment: 0.0
CASRN: 462-06-6, SMILES: Fc1ccccc1, dipole_moment: 1.6
CASRN: 463-49-0, SMILES: [CH2]=[C]=[CH2], dipole_moment: 0.0
CASRN: 463-51-4, SMILES: C=C=O, dipole_moment: 1.42
CASRN: 463-56-9, SMILES: SC#N, dipole_moment: 3.604
CASRN: 463-57-0, SMILES: OCO, dipole_moment: 3.0623
CASRN: 463-58-1, SMILES: O=C=S, dipole_moment: 0.71
CASRN: 463-82-1, SMILES: CC(C)(C)C, dipole_moment: 0.0
CASRN: 464-06-2, SMILES: CC(C)C(C)(C)C, dipole_moment: 0.0
CASRN: 493-01-6, SMILES: C1CCC2CCCCC2C1, dipole_moment: 0.0
CASRN: 493-02-7, SMILES: C1CCC2CCCCC2C1, dipole_moment: 0.0
CASRN: 497-20-1, SMILES: C=C1C=CC=C1, dipole_moment: 0.42
CASRN: 503-17-3, SMILES: CC#CC, dipole_moment: 0.0
CASRN: 503-30-0, SMILES: C1COC1, dipole_moment: 1.94
CASRN: 503-74-2, SMILES: CC(C)CC(O)=O, dipole_moment: 0.629984051036683
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/504-64-3/smiles
CASRN: 504-64-3, SMILES: None, dipole_moment: 0.0
CASRN: 505-22-6, SMILES: C1COCOC1, dipole_moment: 2.06
CASRN: 506-66-1, SMILES: [Be]=C=[Be], dipole_moment: 3.2796
CASRN: 506-68-3, SMILES: BrC#N, dipole_moment: 3.1966
CASRN: 506-77-4, SMILES: ClC#N, dipole_moment: 2.82
CASRN: 506-80-9, SMILES: [Se]=C=[Se], dipole_moment: 2.8152
CASRN: 506-85-4, SMILES: ON=C, dipole_moment: 3.06
CASRN: 507-20-0, SMILES: CC(C)(C)Cl, dipole_moment: 2.13
CASRN: 513-35-9, SMILES: CC=C(C)C, dipole_moment: 0.338765574222638
CASRN: 534-22-5, SMILES: Cc1occc1, dipole_moment: 0.7
CASRN: 536-74-3, SMILES: C#Cc1ccccc1, dipole_moment: 0.66
CASRN: 538-93-2, SMILES: CC(C)Cc1ccccc1, dipole_moment: 0.3
CASRN: 540-54-5, SMILES: CCCCl, dipole_moment: 2.05
CASRN: 540-67-0, SMILES: CCOC, dipole_moment: 1.23
CASRN: 540-88-5, SMILES: CC(=O)OC(C)(C)C, dipole_moment: 1.91
CASRN: 542-55-2, SMILES: CC(C)COC=O, dipole_moment: 1.89001211161876
CASRN: 542-92-7, SMILES: C1C=CC=C1, dipole_moment: 0.42
CASRN: 543-59-9, SMILES: CCCCCCl, dipole_moment: 2.16
CASRN: 544-76-3, SMILES: CCCCCCCCCCCCCCCC, dipole_moment: 0.0
CASRN: 544-92-3, SMILES: [Cu+].[C-]#N, dipole_moment: 0.6239
CASRN: 547-63-7, SMILES: COC(=O)C(C)C, dipole_moment: 2.0
CASRN: 554-12-1, SMILES: CCC(=O)OC, dipole_moment: 1.7
CASRN: 557-91-5, SMILES: CC(Br)Br, dipole_moment: 2.141
CASRN: 557-98-2, SMILES: CC(Cl)=C, dipole_moment: 1.66
CASRN: 557-99-3, SMILES: CC(F)=O, dipole_moment: 2.96
CASRN: 558-13-4, SMILES: BrC(Br)(Br)Br, dipole_moment: 0.0
CASRN: 562-49-2, SMILES: CCC(C)(C)CC, dipole_moment: 0.0
CASRN: 563-45-1, SMILES: CC(C)C=C, dipole_moment: 0.320778021609046
CASRN: 563-46-2, SMILES: CCC(C)=C, dipole_moment: 0.509647324051756
CASRN: 563-80-4, SMILES: CC(C)C(C)=O, dipole_moment: 2.76999316473001
CASRN: 565-59-3, SMILES: CCC(C)C(C)C, dipole_moment: 0.0
CASRN: 565-80-0, SMILES: CC(C)C(=O)C(C)C, dipole_moment: 2.731
CASRN: 583-58-4, SMILES: Cc1ccncc1C, dipole_moment: 1.9
CASRN: 583-61-9, SMILES: Cc1cccnc1C, dipole_moment: 2.2
CASRN: 589-34-4, SMILES: CCCC(C)CC, dipole_moment: 0.0
CASRN: 589-93-5, SMILES: Cc1ccc(C)nc1, dipole_moment: 2.2
CASRN: 590-01-2, SMILES: CCCCOC(=O)CC, dipole_moment: 1.8
CASRN: 590-18-1, SMILES: C\C=C/C, dipole_moment: 0.26
CASRN: 590-19-2, SMILES: C[CH]=[C]=[CH2], dipole_moment: 0.4
CASRN: 590-35-2, SMILES: CCCC(C)(C)C, dipole_moment: 0.0
CASRN: 591-22-0, SMILES: Cc1cncc(C)c1, dipole_moment: 2.6
CASRN: 591-76-4, SMILES: CCCCC(C)C, dipole_moment: 0.0
CASRN: 591-78-6, SMILES: CCCCC(C)=O, dipole_moment: 2.67999544315334
CASRN: 592-41-6, SMILES: CCCCC=C, dipole_moment: 0.339994723651233
CASRN: 592-57-4, SMILES: C1CC=CC=C1, dipole_moment: 0.44
CASRN: 592-76-7, SMILES: CCCCCC=C, dipole_moment: 0.3
CASRN: 592-84-7, SMILES: CCCCOC=O, dipole_moment: 1.91867227878308
CASRN: 593-45-3, SMILES: CCCCCCCCCCCCCCCCCC, dipole_moment: 0.0
CASRN: 593-51-1, SMILES: [Cl-].C[NH3+], dipole_moment: 6.5726
CASRN: 593-53-3, SMILES: CF, dipole_moment: 1.85
CASRN: 593-54-4, SMILES: CP, dipole_moment: 1.1
CASRN: 593-60-2, SMILES: BrC=C, dipole_moment: 1.42
CASRN: 593-61-3, SMILES: BrC#C, dipole_moment: 0.6312
CASRN: 593-63-5, SMILES: ClC#C, dipole_moment: 0.44
CASRN: 593-67-9, SMILES: NC=C, dipole_moment: 0.6548
CASRN: 593-68-0, SMILES: CCP, dipole_moment: 1.6439
CASRN: 593-70-4, SMILES: FCCl, dipole_moment: 1.82
CASRN: 593-75-9, SMILES: C[N+]#[C-], dipole_moment: 3.85
CASRN: 593-77-1, SMILES: [H+].[Cl-].CNO, dipole_moment: 2.6412
CASRN: 593-78-2, SMILES: COCl, dipole_moment: 1.9514
CASRN: 593-79-3, SMILES: C[Se]C, dipole_moment: 1.9099
CASRN: 593-88-4, SMILES: C[As](C)C, dipole_moment: 0.86
CASRN: 594-20-7, SMILES: CC(C)(Cl)Cl, dipole_moment: 2.27
CASRN: 598-25-4, SMILES: C[C](C)=[C]=[CH2], dipole_moment: 0.55
CASRN: 598-58-3, SMILES: CO[N+]([O-])=O, dipole_moment: 3.08
CASRN: 611-14-3, SMILES: CCc1ccccc1C, dipole_moment: 0.56
CASRN: 616-42-2, SMILES: CO[S](=O)OC, dipole_moment: 3.09
CASRN: 617-78-7, SMILES: CCC(CC)CC, dipole_moment: 0.0
CASRN: 622-96-8, SMILES: CCc1ccc(C)cc1, dipole_moment: 0.0
CASRN: 623-42-7, SMILES: CCCC(=O)OC, dipole_moment: 1.71999976016597
CASRN: 624-64-6, SMILES: C\C=C\C, dipole_moment: 0.0
CASRN: 624-72-6, SMILES: FCCF, dipole_moment: 2.671
CASRN: 624-91-9, SMILES: CON=O, dipole_moment: 2.05
CASRN: 625-58-1, SMILES: CCO[N+]([O-])=O, dipole_moment: 3.39
CASRN: 627-05-4, SMILES: CCCC[N+]([O-])=O, dipole_moment: 3.4
CASRN: 627-19-0, SMILES: CCCC#C, dipole_moment: 0.85
CASRN: 627-20-3, SMILES: CC\C=C/C, dipole_moment: 0.28
CASRN: 627-26-9, SMILES: C/C=C/C#N, dipole_moment: 4.5
CASRN: 627-51-0, SMILES: C=CSC=C, dipole_moment: 1.2
CASRN: 628-32-0, SMILES: CCCOCC, dipole_moment: 1.15998728879615
CASRN: 628-76-2, SMILES: ClCCCCCCl, dipole_moment: 2.36
CASRN: 628-81-9, SMILES: CCCCOCC, dipole_moment: 1.22000575601684
CASRN: 629-11-8, SMILES: OCCCCCCO, dipole_moment: 2.5
CASRN: 629-50-5, SMILES: CCCCCCCCCCCCC, dipole_moment: 0.0
CASRN: 629-59-4, SMILES: CCCCCCCCCCCCCC, dipole_moment: 0.0
CASRN: 629-62-9, SMILES: CCCCCCCCCCCCCCC, dipole_moment: 0.0
CASRN: 629-78-7, SMILES: CCCCCCCCCCCCCCCCC, dipole_moment: 0.0
CASRN: 629-92-5, SMILES: CCCCCCCCCCCCCCCCCCC, dipole_moment: 0.0
CASRN: 629-94-7, SMILES: CCCCCCCCCCCCCCCCCCCCC, dipole_moment: 0.0
CASRN: 629-97-0, SMILES: CCCCCCCCCCCCCCCCCCCCCC, dipole_moment: 0.0
CASRN: 630-08-0, SMILES: [C-]#[O+], dipole_moment: 0.11
CASRN: 630-18-2, SMILES: CC(C)(C)C#N, dipole_moment: 3.95
CASRN: 638-49-3, SMILES: CCCCCOC=O, dipole_moment: 1.9
CASRN: 638-67-5, SMILES: CCCCCCCCCCCCCCCCCCCCCCC, dipole_moment: 0.0
CASRN: 646-04-8, SMILES: CC\C=C\C, dipole_moment: 0.0
CASRN: 646-31-1, SMILES: CCCCCCCCCCCCCCCCCCCCCCCC, dipole_moment: 0.0
CASRN: 676-49-3, SMILES: C[2H], dipole_moment: 0.0176
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/676-55-1/smiles
CASRN: 676-55-1, SMILES: None, dipole_moment: 0.0123
CASRN: 676-59-5, SMILES: CPC, dipole_moment: 1.7033
CASRN: 678-26-2, SMILES: FC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F, dipole_moment: 0.0
CASRN: 689-97-4, SMILES: C=CC#C, dipole_moment: 0.22
CASRN: 691-37-2, SMILES: CC(C)CC=C, dipole_moment: 0.265616193594033
CASRN: 693-02-7, SMILES: CCCCC#C, dipole_moment: 0.83
CASRN: 693-86-7, SMILES: C=CC1CC1, dipole_moment: 0.5
CASRN: 694-01-9, SMILES: C1C=CC2CC12, dipole_moment: 0.29
CASRN: 765-30-0, SMILES: NC1CC1, dipole_moment: 1.19
CASRN: 765-43-5, SMILES: CC(=O)C1CC1, dipole_moment: 2.62
CASRN: 822-35-5, SMILES: C1CC=C1, dipole_moment: 0.13
CASRN: 865-36-1, SMILES: C=S, dipole_moment: 1.65
CASRN: 917-54-4, SMILES: [Li+].[CH3-], dipole_moment: 2.356
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/992-94-9/smiles
CASRN: 992-94-9, SMILES: None, dipole_moment: 0.74
CASRN: 1066-27-9, SMILES: [SiH3]C#C, dipole_moment: 0.3521
CASRN: 1066-43-9, SMILES: C[Si], dipole_moment: 0.5368
CASRN: 1070-71-9, SMILES: C#CC#N, dipole_moment: 3.72
CASRN: 1072-43-1, SMILES: CC1CS1, dipole_moment: 1.95
CASRN: 1111-74-6, SMILES: C[SiH2]C, dipole_moment: 0.75
CASRN: 1120-21-4, SMILES: CCCCCCCCCCC, dipole_moment: 0.0
CASRN: 1120-56-5, SMILES: [CH2]=[C]1C[CH2]C1, dipole_moment: 0.51
CASRN: 1190-76-7, SMILES: C\C=C/C#N, dipole_moment: 4.08
CASRN: 1191-95-3, SMILES: O=C1CCC1, dipole_moment: 2.99
CASRN: 1303-00-0, SMILES: [Ga]#[As], dipole_moment: 2.9749
CASRN: 1304-56-9, SMILES: [Be]=O, dipole_moment: 7.4836
CASRN: 1305-78-8, SMILES: O=[Ca], dipole_moment: 7.2202
CASRN: 1305-84-6, SMILES: [Ca]=[Se], dipole_moment: 3.6912
CASRN: 1309-48-4, SMILES: O=[Mg], dipole_moment: 7.5578
CASRN: 1310-53-8, SMILES: O=[Ge]=O, dipole_moment: 3.0658
CASRN: 1310-66-3, SMILES: [Li+].O.[OH-], dipole_moment: 4.76
CASRN: 1313-99-1, SMILES: O=[Ni], dipole_moment: 7.4636
CASRN: 1314-05-2, SMILES: [Ni]=[Se], dipole_moment: 6.1658
CASRN: 1314-13-2, SMILES: O=[Zn], dipole_moment: 5.4411
CASRN: 1314-98-3, SMILES: S=[Zn], dipole_moment: 6.2041
CASRN: 1317-37-9, SMILES: S=[Fe], dipole_moment: 1.2889
CASRN: 1317-38-0, SMILES: O=[Cu], dipole_moment: 4.5
CASRN: 1317-66-4, SMILES: [Fe++].[S-][S-], dipole_moment: 0.0
CASRN: 1333-74-0, SMILES: [H], dipole_moment: 0.0
CASRN: 1336-21-6, SMILES: [NH4+].[OH-], dipole_moment: 5.6302
CASRN: 1345-25-1, SMILES: O=[Fe], dipole_moment: 3.0224
CASRN: 1449-64-5, SMILES: C[GeH2]C, dipole_moment: 0.5341
CASRN: 1449-65-6, SMILES: C[GeH3], dipole_moment: 0.64
CASRN: 1455-13-6, SMILES: CO[2H], dipole_moment: 1.8082
CASRN: 1493-02-3, SMILES: FC=O, dipole_moment: 2.02
CASRN: 1495-50-7, SMILES: FC#N, dipole_moment: 2.17
CASRN: 1511-62-2, SMILES: FC(F)Br, dipole_moment: 1.5
CASRN: 1517-53-9, SMILES: C=C, dipole_moment: 0.0059
CASRN: 1574-41-0, SMILES: C\C=C/C=C, dipole_moment: 0.5
CASRN: 1590-87-0, SMILES: [SiH2]=[SiH2], dipole_moment: 0.0
CASRN: 1603-84-5, SMILES: O=C=[Se], dipole_moment: 0.75
CASRN: 1605-72-7, SMILES: ClCCl, dipole_moment: 0.6084
CASRN: 1613-51-0, SMILES: C1CCSCC1, dipole_moment: 1.78
CASRN: 1630-77-9, SMILES: F\C=C/F, dipole_moment: 2.42
CASRN: 1630-78-0, SMILES: F\C=C\F, dipole_moment: 0.55
CASRN: 1630-94-0, SMILES: CC1(C)CC1, dipole_moment: 0.14
CASRN: 1634-09-9, SMILES: CCCCc1cccc2ccccc12, dipole_moment: 0.69
CASRN: 1640-89-7, SMILES: CCC1CCCC1, dipole_moment: 0.0
CASRN: 1708-29-8, SMILES: C1OCC=C1, dipole_moment: 1.32
CASRN: 1708-32-3, SMILES: C1SCC=C1, dipole_moment: 1.75
CASRN: 1738-36-9, SMILES: COCC#N, dipole_moment: 2.98
CASRN: 1747-99-5, SMILES: CC[GeH3], dipole_moment: 0.4489
CASRN: 1761-67-7, SMILES: CN=C, dipole_moment: 1.53
CASRN: 2004-69-5, SMILES: C/C=C/C#C, dipole_moment: 0.66
CASRN: 2004-70-8, SMILES: C/C=C/C=C, dipole_moment: 0.58
CASRN: 2053-29-4, SMILES: C=N, dipole_moment: 2.0
CASRN: 2074-87-5, SMILES: NCCN, dipole_moment: 1.45
CASRN: 2154-59-8, SMILES: FCF, dipole_moment: 0.46
CASRN: 2171-96-2, SMILES: CO[SiH3], dipole_moment: 1.5306
CASRN: 2210-34-6, SMILES: C(#C)[2H], dipole_moment: 0.028
CASRN: 2229-07-4, SMILES: [CH3], dipole_moment: 0.0
CASRN: 2314-97-8, SMILES: FC(F)(F)I, dipole_moment: 1.05
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/2465-56-7/smiles
CASRN: 2465-56-7, SMILES: None, dipole_moment: 1.9466
CASRN: 2533-82-6, SMILES: C[As]=S, dipole_moment: 1.8805
CASRN: 2551-62-4, SMILES: F[S](F)(F)(F)(F)F, dipole_moment: 0.0
CASRN: 2565-30-2, SMILES: ClC=O, dipole_moment: 2.4735
CASRN: 2644-70-4, SMILES: [H+].[H+].[Cl-].[Cl-].NN, dipole_moment: 3.131
CASRN: 2680-00-4, SMILES: C(=C)[2H], dipole_moment: 0.001
CASRN: 2696-92-6, SMILES: ClN=O, dipole_moment: 1.9
CASRN: 2699-79-8, SMILES: F[S](F)(=O)=O, dipole_moment: 1.12
CASRN: 2713-09-9, SMILES: FC#C, dipole_moment: 0.73
CASRN: 2781-85-3, SMILES: C1C=C1, dipole_moment: 0.45
CASRN: 2809-69-0, SMILES: CC#CC#CC, dipole_moment: 0.0
CASRN: 2944-05-0, SMILES: C=S, dipole_moment: 1.98
CASRN: 3017-23-0, SMILES: C(#N)[2H], dipole_moment: 3.2179
CASRN: 3031-73-0, SMILES: COO, dipole_moment: 3.198
CASRN: 3100-04-7, SMILES: CC1=CC1, dipole_moment: 0.84
CASRN: 3129-90-6, SMILES: N=C=S, dipole_moment: 2.1076
CASRN: 3352-57-6, SMILES: [OH-], dipole_moment: 1.66
CASRN: 3400-09-7, SMILES: ClNCl, dipole_moment: 1.7284
CASRN: 3618-05-1, SMILES: NN, dipole_moment: 0.0312
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/3889-75-6/smiles
CASRN: 3889-75-6, SMILES: None, dipole_moment: 0.64
CASRN: 4109-96-0, SMILES: Cl[SiH2]Cl, dipole_moment: 1.13
CASRN: 4170-30-3, SMILES: C\C=C/C=O, dipole_moment: 3.72
CASRN: 4960-86-5, SMILES: C(Cl)[2H], dipole_moment: 2.2179
CASRN: 5009-27-8, SMILES: O=C1CC1, dipole_moment: 2.67
CASRN: 5164-35-2, SMILES: C1C2C=CC12, dipole_moment: 0.4
CASRN: 5188-07-8, SMILES: [Na+].C[S-], dipole_moment: 4.4426
CASRN: 5194-50-3, SMILES: C/C=C/C=C\C, dipole_moment: 0.309
CASRN: 5470-11-1, SMILES: [H+].[Cl-].NO, dipole_moment: 2.4225
CASRN: 5505-49-7, SMILES: C(C)[2H], dipole_moment: 0.026
CASRN: 5813-48-9, SMILES: CSCl, dipole_moment: 2.8279
CASRN: 5951-19-9, SMILES: S=C=[Se], dipole_moment: 2.8635
CASRN: 6004-44-0, SMILES: CCOC(=O)C(=CNc1cc(Cl)ccc1OC)C(=O)OCC, dipole_moment: 1.79
CASRN: 6142-73-0, SMILES: C=C1CC1, dipole_moment: 0.4
CASRN: 6154-14-9, SMILES: CNCl, dipole_moment: 2.2466
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/6251-26-9/smiles
CASRN: 6251-26-9, SMILES: None, dipole_moment: 2.2824
CASRN: 6303-21-5, SMILES: O[PH+]=O, dipole_moment: 1.8637
CASRN: 6569-82-0, SMILES: CP=C, dipole_moment: 1.8266
CASRN: 6725-64-0, SMILES: SCS, dipole_moment: 2.236
CASRN: 6746-94-7, SMILES: C#CC1CC1, dipole_moment: 0.89
CASRN: 6829-52-3, SMILES: C#P, dipole_moment: 0.39
CASRN: 6851-93-0, SMILES: CC=S, dipole_moment: 2.1903
CASRN: 6914-07-4, SMILES: C=N, dipole_moment: 3.05
CASRN: 6915-15-7, SMILES: OC(CC(O)=O)C(O)=O, dipole_moment: 3.118
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/7282-87-3/smiles
CASRN: 7282-87-3, SMILES: None, dipole_moment: 0.1497
CASRN: 7429-90-5, SMILES: [Al], dipole_moment: 0.0
CASRN: 7439-89-6, SMILES: [Fe], dipole_moment: 0.0
CASRN: 7439-90-9, SMILES: [Kr], dipole_moment: 0.0
CASRN: 7439-93-2, SMILES: [Li], dipole_moment: 0.0
CASRN: 7439-95-4, SMILES: [Mg++], dipole_moment: 0.0
CASRN: 7440-01-9, SMILES: [Ne], dipole_moment: 0.0
CASRN: 7440-02-0, SMILES: [Ni], dipole_moment: 0.0
CASRN: 7440-21-3, SMILES: [Si], dipole_moment: 0.0
CASRN: 7440-23-5, SMILES: [Na+], dipole_moment: 0.0
CASRN: 7440-32-6, SMILES: [Ti], dipole_moment: 0.0
CASRN: 7440-37-1, SMILES: [Ar], dipole_moment: 0.0
CASRN: 7440-41-7, SMILES: [Be], dipole_moment: 0.0
CASRN: 7440-42-8, SMILES: [B], dipole_moment: 0.0
CASRN: 7440-44-0, SMILES: [C], dipole_moment: 0.0
CASRN: 7440-47-3, SMILES: [Cr], dipole_moment: 0.0
CASRN: 7440-56-4, SMILES: [Ge], dipole_moment: 0.0
CASRN: 7440-59-7, SMILES: [He], dipole_moment: 0.0
CASRN: 7440-66-6, SMILES: [Zn], dipole_moment: 0.0
CASRN: 7440-70-2, SMILES: [Ca], dipole_moment: 0.0
CASRN: 7446-08-4, SMILES: O=[Se]=O, dipole_moment: 2.62
CASRN: 7446-09-5, SMILES: O=[S]=O, dipole_moment: 1.63
CASRN: 7446-11-9, SMILES: O=[S](=O)=O, dipole_moment: 0.0
CASRN: 7446-34-6, SMILES: S=[Se], dipole_moment: 0.668
CASRN: 7446-70-0, SMILES: [Al+3].[Cl-].[Cl-].[Cl-], dipole_moment: 0.0
CASRN: 7447-40-7, SMILES: [Cl-].[K+], dipole_moment: 10.24
CASRN: 7447-41-8, SMILES: [Li+].[Cl-], dipole_moment: 7.13
CASRN: 7550-35-8, SMILES: [Li+].[Br-], dipole_moment: 7.23
CASRN: 7580-67-8, SMILES: [Li+].[H-], dipole_moment: 5.88
CASRN: 7616-94-6, SMILES: F[Cl](=O)(=O)=O, dipole_moment: 0.02
CASRN: 7631-86-9, SMILES: O=[Si]=O, dipole_moment: 0.0
CASRN: 7637-07-2, SMILES: FB(F)F, dipole_moment: 0.0
CASRN: 7642-09-3, SMILES: CC\C=C/CC, dipole_moment: 0.339
CASRN: 7647-01-0, SMILES: [H+].[Cl-], dipole_moment: 1.08
CASRN: 7647-14-5, SMILES: [Na+].[Cl-], dipole_moment: 9.0
CASRN: 7647-15-6, SMILES: [Na+].[Br-], dipole_moment: 9.12
CASRN: 7647-19-0, SMILES: F[P](F)(F)(F)F, dipole_moment: 0.0
CASRN: 7664-39-3, SMILES: [H+].[F-], dipole_moment: 1.82
CASRN: 7664-41-7, SMILES: N, dipole_moment: 1.47
CASRN: 7664-93-9, SMILES: O[S](O)(=O)=O, dipole_moment: 2.72
CASRN: 7681-49-4, SMILES: [F-].[Na+], dipole_moment: 8.16
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/7693-27-8/smiles
CASRN: 7693-27-8, SMILES: None, dipole_moment: 0.0
CASRN: 7697-37-2, SMILES: O[N+]([O-])=O, dipole_moment: 2.17
CASRN: 7698-05-7, SMILES: [1H+].[Cl-], dipole_moment: 1.5745
CASRN: 7704-34-9, SMILES: S, dipole_moment: 0.0
CASRN: 7719-09-7, SMILES: Cl[S](Cl)=O, dipole_moment: 1.45
CASRN: 7719-12-2, SMILES: ClP(Cl)Cl, dipole_moment: 0.78
CASRN: 7722-84-1, SMILES: OO, dipole_moment: 1.77
CASRN: 7723-14-0, SMILES: [P], dipole_moment: 0.0
CASRN: 7726-95-6, SMILES: [Br-], dipole_moment: 0.0
CASRN: 7727-37-9, SMILES: N#N, dipole_moment: 0.0
CASRN: 7732-18-5, SMILES: O, dipole_moment: 1.85
CASRN: 7758-02-3, SMILES: [K+].[Br-], dipole_moment: 10.6
CASRN: 7758-89-6, SMILES: [Cl-].[Cu+], dipole_moment: 6.4955
CASRN: 7782-39-0, SMILES: [2H][2H], dipole_moment: 0.0
CASRN: 7782-41-4, SMILES: FF, dipole_moment: 0.0
CASRN: 7782-44-7, SMILES: O=O, dipole_moment: 0.0
CASRN: 7782-49-2, SMILES: [Se], dipole_moment: 0.0
CASRN: 7782-50-5, SMILES: ClCl, dipole_moment: 0.0
CASRN: 7782-65-2, SMILES: [GeH4], dipole_moment: 0.063
CASRN: 7782-89-0, SMILES: [Li+].[NH2-], dipole_moment: 1.0182
CASRN: 7782-92-5, SMILES: [NH2-].[Na+], dipole_moment: 0.0616
CASRN: 7783-06-4, SMILES: S, dipole_moment: 0.97
CASRN: 7783-07-5, SMILES: [Se], dipole_moment: 0.63
CASRN: 7783-41-7, SMILES: FOF, dipole_moment: 0.3
CASRN: 7783-44-0, SMILES: FOOF, dipole_moment: 1.44
CASRN: 7783-54-2, SMILES: FN(F)F, dipole_moment: 0.23
CASRN: 7783-55-3, SMILES: FP(F)F, dipole_moment: 1.03
CASRN: 7783-60-0, SMILES: F[S](F)(F)F, dipole_moment: 0.63
CASRN: 7783-61-1, SMILES: F[Si](F)(F)F, dipole_moment: 0.0
CASRN: 7783-81-5, SMILES: [F]|[U](|[F])(|[F])(|[F])(|[F])|[F], dipole_moment: 0.0
CASRN: 7784-18-1, SMILES: [F-].[F-].[F-].[Al+3], dipole_moment: 0.0
CASRN: 7784-21-6, SMILES: [H-].[Al+3], dipole_moment: 0.0
CASRN: 7784-35-2, SMILES: F[As](F)F, dipole_moment: 2.59
CASRN: 7784-42-1, SMILES: [As], dipole_moment: 0.2
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/7787-52-2/smiles
CASRN: 7787-52-2, SMILES: None, dipole_moment: 0.0
CASRN: 7787-70-4, SMILES: [Cu+].[Br-], dipole_moment: 5.5811
CASRN: 7787-71-5, SMILES: F[Br](F)F, dipole_moment: 1.0
CASRN: 7789-20-0, SMILES: O([2H])[2H], dipole_moment: 1.85
CASRN: 7789-23-3, SMILES: [F-].[K+], dipole_moment: 8.56
CASRN: 7789-24-4, SMILES: [Li+].[F-], dipole_moment: 6.28
CASRN: 7789-25-5, SMILES: FN=O, dipole_moment: 1.81
CASRN: 7789-30-2, SMILES: F[Br](F)(F)(F)F, dipole_moment: 1.51
CASRN: 7789-33-5, SMILES: BrI, dipole_moment: 0.74
CASRN: 7789-60-8, SMILES: BrP(Br)Br, dipole_moment: 0.61
CASRN: 7790-89-8, SMILES: FCl, dipole_moment: 0.88
CASRN: 7790-91-2, SMILES: F[Cl](F)F, dipole_moment: 0.6
CASRN: 7790-92-3, SMILES: OCl, dipole_moment: 1.3
CASRN: 7791-25-5, SMILES: Cl[S](Cl)(=O)=O, dipole_moment: 1.81
CASRN: 7803-49-8, SMILES: NO, dipole_moment: 0.59
CASRN: 7803-51-2, SMILES: P, dipole_moment: 0.58
CASRN: 7803-62-5, SMILES: [Si], dipole_moment: 0.0
CASRN: 10022-50-1, SMILES: [O-][N+](F)=O, dipole_moment: 0.5
CASRN: 10024-97-2, SMILES: [N-]=[N+]=O, dipole_moment: 0.17
CASRN: 10025-78-2, SMILES: Cl[Si](Cl)Cl, dipole_moment: 0.86
CASRN: 10026-04-7, SMILES: Cl[Si](Cl)(Cl)Cl, dipole_moment: 0.0
CASRN: 10026-13-8, SMILES: Cl[P](Cl)(Cl)(Cl)Cl, dipole_moment: 0.0
CASRN: 10028-15-6, SMILES: O=[O]=O, dipole_moment: 0.53
CASRN: 10028-17-8, SMILES: [3H][3H], dipole_moment: 0.0
CASRN: 10034-85-2, SMILES: [H+].[I-], dipole_moment: 0.45
CASRN: 10035-10-6, SMILES: [H+].[Br-], dipole_moment: 0.83
CASRN: 10036-47-2, SMILES: FN(F)N(F)F, dipole_moment: 0.3
CASRN: 10043-11-5, SMILES: B#N, dipole_moment: 4.5088
CASRN: 10049-04-4, SMILES: [O-][Cl]=O, dipole_moment: 1.78
CASRN: 10049-06-6, SMILES: [Cl-].[Cl-].[Ti++], dipole_moment: 0.7316
CASRN: 10060-11-4, SMILES: [Cl-].[Cl-].[Ge++], dipole_moment: 1.8529
CASRN: 10097-28-6, SMILES: [O+]#[Si-], dipole_moment: 3.1
CASRN: 10102-43-9, SMILES: N=O, dipole_moment: 0.15
CASRN: 10102-44-0, SMILES: [O-][N+]=O, dipole_moment: 0.32
CASRN: 10218-87-8, SMILES: CNBr, dipole_moment: 2.1622
CASRN: 10294-34-5, SMILES: ClB(Cl)Cl, dipole_moment: 0.0
CASRN: 10405-27-3, SMILES: FNF, dipole_moment: 1.9
CASRN: 10544-72-6, SMILES: [O-][N+](=O)[N+]([O-])=O, dipole_moment: 0.0
CASRN: 10544-73-7, SMILES: [O-][N+](=O)N=O, dipole_moment: 2.12
CASRN: 10545-99-0, SMILES: ClSCl, dipole_moment: 0.36
CASRN: 10599-90-3, SMILES: NCl, dipole_moment: 2.2982
CASRN: 11062-77-4, SMILES: [O][O-], dipole_moment: 0.0
CASRN: 11113-63-6, SMILES: C.FF, dipole_moment: 4.1191
CASRN: 12004-45-4, SMILES: [AlH2+].[AlH2+].[S--], dipole_moment: 2.179
CASRN: 12005-95-7, SMILES: [Mn]#[As], dipole_moment: 4.7914
CASRN: 12013-55-7, SMILES: [SiH2]=[Ca], dipole_moment: 5.2589
CASRN: 12018-00-7, SMILES: O=[Cr], dipole_moment: 6.5612
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/12018-08-5/smiles
CASRN: 12018-08-5, SMILES: None, dipole_moment: 3.3664
CASRN: 12022-99-0, SMILES: [SiH2]=[Fe]=[SiH2], dipole_moment: 3.4683
CASRN: 12024-20-3, SMILES: [GaH2]O[GaH2], dipole_moment: 7.0472
CASRN: 12025-34-2, SMILES: S=[Ge]=S, dipole_moment: 5.678
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/12030-83-0/smiles
CASRN: 12030-83-0, SMILES: None, dipole_moment: 3.45
CASRN: 12033-56-6, SMILES: [N]=S, dipole_moment: 1.81
CASRN: 12033-76-0, SMILES: ON=[SiH2], dipole_moment: 2.5698
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/12035-57-3/smiles
CASRN: 12035-57-3, SMILES: None, dipole_moment: 3.3756
CASRN: 12039-13-3, SMILES: S=[Ti]=S, dipole_moment: 8.198
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/12039-70-2/smiles
CASRN: 12039-70-2, SMILES: None, dipole_moment: 3.6023
CASRN: 12039-83-7, SMILES: [SiH2]=[Ti]=[SiH2], dipole_moment: 3.256
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/12041-50-8/smiles
CASRN: 12041-50-8, SMILES: None, dipole_moment: 3.6214
CASRN: 12045-15-7, SMILES: B#[Mn], dipole_moment: 4.059
CASRN: 12045-25-9, SMILES: SB=S, dipole_moment: 5.7183
CASRN: 12045-27-1, SMILES: B#[V], dipole_moment: 5.5721
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/12049-73-9/smiles
CASRN: 12049-73-9, SMILES: None, dipole_moment: 2.0486
CASRN: 12053-13-3, SMILES: [Cr]=[Se], dipole_moment: 10.5493
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/12056-29-0/smiles
CASRN: 12056-29-0, SMILES: None, dipole_moment: 2.67
CASRN: 12061-70-0, SMILES: OF, dipole_moment: 0.0
CASRN: 12063-98-8, SMILES: P#[Ga], dipole_moment: 3.5585
CASRN: 12065-11-1, SMILES: [Se]=[Ge]=[Se], dipole_moment: 4.519
CASRN: 12066-53-4, SMILES: P#[V], dipole_moment: 3.1738
CASRN: 12067-45-7, SMILES: [Se]=[Ti]=[Se], dipole_moment: 2.2643
CASRN: 12070-15-4, SMILES: CC, dipole_moment: 0.0
CASRN: 12075-35-3, SMILES: [CH2]=[C]=[CH2], dipole_moment: 0.0
CASRN: 12124-99-1, SMILES: [NH4+].[SH-], dipole_moment: 3.8925
CASRN: 12125-02-9, SMILES: [NH4+].[Cl-], dipole_moment: 3.427
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/12135-07-8/smiles
CASRN: 12135-07-8, SMILES: None, dipole_moment: 1.5899
CASRN: 12136-60-6, SMILES: [Li+].[Li+].[Se--], dipole_moment: 7.766
CASRN: 12137-20-1, SMILES: O=[Ti], dipole_moment: 0.5539
CASRN: 12142-77-7, SMILES: [Li+].[Li+].[O--], dipole_moment: 6.84
CASRN: 12185-09-0, SMILES: P#P, dipole_moment: 0.0
CASRN: 12185-10-3, SMILES: P12P3P1P23, dipole_moment: 0.0
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/12190-75-9/smiles
CASRN: 12190-75-9, SMILES: None, dipole_moment: 0.5903
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/12200-64-5/smiles
CASRN: 12200-64-5, SMILES: None, dipole_moment: 5.3868
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/12201-89-7/smiles
CASRN: 12201-89-7, SMILES: None, dipole_moment: 0.0
CASRN: 12211-52-8, SMILES: [NH4+].[C-]#N, dipole_moment: 6.4553
CASRN: 12232-25-6, SMILES: [Be]=[Se], dipole_moment: 6.6389
CASRN: 12251-90-0, SMILES: [AlH2]S, dipole_moment: 1.6242
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/12265-14-4/smiles
CASRN: 12265-14-4, SMILES: None, dipole_moment: 2.3624
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/12333-49-2/smiles
CASRN: 12333-49-2, SMILES: None, dipole_moment: 0.46
CASRN: 12357-66-3, SMILES: N#[NH+], dipole_moment: 3.1411
CASRN: 12385-13-6, SMILES: [H][H], dipole_moment: 0.0
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/12434-84-3/smiles
CASRN: 12434-84-3, SMILES: None, dipole_moment: 5.344
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/12504-41-5/smiles
CASRN: 12504-41-5, SMILES: None, dipole_moment: 1.73
CASRN: 12589-02-5, SMILES: B[SiH3], dipole_moment: 0.719
CASRN: 12596-80-4, SMILES: [O]O[O-], dipole_moment: 1.0748
CASRN: 12597-03-4, SMILES: S=[S]=S, dipole_moment: 0.3422
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/12597-05-6/smiles
CASRN: 12597-05-6, SMILES: None, dipole_moment: 1.2554
CASRN: 12597-35-2, SMILES: [SiH2]=[SiH2], dipole_moment: 0.0
CASRN: 12601-93-3, SMILES: P=C=P, dipole_moment: 1.9119
CASRN: 12602-39-0, SMILES: PC#C, dipole_moment: 0.805
CASRN: 12628-09-0, SMILES: B=C=[SiH2], dipole_moment: 2.9052
CASRN: 13269-52-8, SMILES: CC\C=C\CC, dipole_moment: 0.0
CASRN: 13283-31-3, SMILES: B, dipole_moment: 0.0
CASRN: 13444-87-6, SMILES: BrN=O, dipole_moment: 0.4091
CASRN: 13444-90-1, SMILES: [Cl-]|[NH+]([O-])=O, dipole_moment: 0.53
CASRN: 13445-50-6, SMILES: PP, dipole_moment: 1.7845
CASRN: 13462-88-9, SMILES: [Ni++].[Br-].[Br-], dipole_moment: 10.4556
CASRN: 13463-67-7, SMILES: O=[Ti]=O, dipole_moment: 7.3723
CASRN: 13465-07-1, SMILES: S, dipole_moment: 1.5878
CASRN: 13465-78-6, SMILES: [SiH2]Cl, dipole_moment: 1.31
CASRN: 13478-20-1, SMILES: F[P](F)(F)=O, dipole_moment: 1.76
CASRN: 13478-41-6, SMILES: [F-].[Cu+], dipole_moment: 5.77
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/13517-00-5/smiles
CASRN: 13517-00-5, SMILES: None, dipole_moment: 3.372
CASRN: 13517-11-8, SMILES: OBr, dipole_moment: 1.38
CASRN: 13536-59-9, SMILES: [1H+].[Br-], dipole_moment: 1.2028
CASRN: 13536-94-2, SMILES: S([2H])[2H], dipole_moment: 1.4264
CASRN: 13537-30-9, SMILES: [F]|[GeH3], dipole_moment: 2.1983
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/13537-33-2/smiles
CASRN: 13537-33-2, SMILES: None, dipole_moment: 1.3
CASRN: 13569-43-2, SMILES: [GeH3]|[Br], dipole_moment: 2.3785
CASRN: 13587-49-0, SMILES: N[2H], dipole_moment: 1.6909
CASRN: 13587-50-3, SMILES: P, dipole_moment: 1.2139
CASRN: 13595-81-8, SMILES: [Al+].[Cl-], dipole_moment: 2.3239
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/13595-82-9/smiles
CASRN: 13595-82-9, SMILES: None, dipole_moment: 1.53
CASRN: 13597-70-1, SMILES: PPP, dipole_moment: 1.2525
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/13598-78-2/smiles
CASRN: 13598-78-2, SMILES: None, dipole_moment: 1.5811
CASRN: 13637-65-5, SMILES: [Cl]|[GeH3], dipole_moment: 2.2686
CASRN: 13701-67-2, SMILES: ClB(Cl)B(Cl)Cl, dipole_moment: 0.0
CASRN: 13709-35-8, SMILES: FSSF, dipole_moment: 1.45
CASRN: 13709-36-9, SMILES: F[Xe]F, dipole_moment: 0.0
CASRN: 13709-61-0, SMILES: F[Xe](F)(F)F, dipole_moment: 0.0
CASRN: 13709-83-6, SMILES: FBF, dipole_moment: 1.5083
CASRN: 13759-10-9, SMILES: S=[Si]=S, dipole_moment: 0.9794
CASRN: 13762-18-0, SMILES: S[2H], dipole_moment: 1.4481
CASRN: 13765-45-2, SMILES: [GeH3], dipole_moment: 0.1695
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/13766-26-2/smiles
CASRN: 13766-26-2, SMILES: None, dipole_moment: 1.27
CASRN: 13766-28-4, SMILES: O=BB=O, dipole_moment: 0.0
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/13767-16-3/smiles
CASRN: 13767-16-3, SMILES: None, dipole_moment: 1.8277
CASRN: 13768-07-5, SMILES: O[As]=O, dipole_moment: 2.6838
CASRN: 13768-60-0, SMILES: BF, dipole_moment: 0.5
CASRN: 13768-94-0, SMILES: Br[SiH2]Br, dipole_moment: 2.0042
CASRN: 13769-36-3, SMILES: [GeH2](|[Br])|[Br], dipole_moment: 2.2488
CASRN: 13773-81-4, SMILES: F[Kr]F, dipole_moment: 2.1951
CASRN: 13774-92-0, SMILES: N, dipole_moment: 1.7756
CASRN: 13775-80-9, SMILES: [H+].[Br-].NN, dipole_moment: 2.7701
CASRN: 13776-62-0, SMILES: FN=NF, dipole_moment: 0.0
CASRN: 13780-23-9, SMILES: [S], dipole_moment: 0.76
CASRN: 13780-28-4, SMILES: N([2H])[2H], dipole_moment: 1.8856
CASRN: 13780-29-5, SMILES: P([2H])[2H], dipole_moment: 1.2189
CASRN: 13812-43-6, SMILES: FN=NF, dipole_moment: 0.16
CASRN: 13814-20-5, SMILES: [F-].[F-].[Ti++], dipole_moment: 9.4166
CASRN: 13814-25-0, SMILES: FSF, dipole_moment: 1.05
CASRN: 13817-04-4, SMILES: N=[S]=O, dipole_moment: 1.4794
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/13817-06-6/smiles
CASRN: 13817-06-6, SMILES: None, dipole_moment: 2.9949
CASRN: 13824-36-7, SMILES: F[SiH2]F, dipole_moment: 2.1993
CASRN: 13827-32-2, SMILES: O=S, dipole_moment: 1.55
CASRN: 13840-88-5, SMILES: [OH2+][B][O-], dipole_moment: 0.0
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/13845-23-3/smiles
CASRN: 13845-23-3, SMILES: None, dipole_moment: 0.7066
CASRN: 13863-41-7, SMILES: ClBr, dipole_moment: 0.52
CASRN: 13863-59-7, SMILES: FBr, dipole_moment: 1.42
CASRN: 13898-47-0, SMILES: [Na+].[O-][Cl]=O, dipole_moment: 4.8764
CASRN: 13940-21-1, SMILES: [SH], dipole_moment: 0.76
CASRN: 13940-63-1, SMILES: [F]|[GeH2]|[F], dipole_moment: 2.61
CASRN: 13952-84-6, SMILES: CCC(C)N, dipole_moment: 1.28
CASRN: 13965-73-6, SMILES: FB(F)B(F)F, dipole_moment: 0.0
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/13965-74-7/smiles
CASRN: 13965-74-7, SMILES: None, dipole_moment: 0.7819
CASRN: 13966-66-0, SMILES: [F-].[Si].[H+], dipole_moment: 1.23
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/13966-78-4/smiles
CASRN: 13966-78-4, SMILES: None, dipole_moment: 2.45
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/13983-20-5/smiles
CASRN: 13983-20-5, SMILES: None, dipole_moment: 0.0
CASRN: 14017-34-6, SMILES: F[Se]F, dipole_moment: 2.3721
CASRN: 14034-79-8, SMILES: OF, dipole_moment: 2.23
CASRN: 14044-97-4, SMILES: [SiH3]S, dipole_moment: 1.6122
CASRN: 14067-04-0, SMILES: [Ti], dipole_moment: 0.0
CASRN: 14127-61-8, SMILES: [Ca++], dipole_moment: 0.0
CASRN: 14312-20-0, SMILES: BrSBr, dipole_moment: 0.2627
CASRN: 14314-42-2, SMILES: O, dipole_moment: 2.1164
CASRN: 14332-28-6, SMILES: N=O, dipole_moment: 1.67
CASRN: 14452-59-6, SMILES: [Li][Li], dipole_moment: 0.0
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/14452-60-9/smiles
CASRN: 14452-60-9, SMILES: None, dipole_moment: 0.0
CASRN: 14452-66-5, SMILES: O=[P], dipole_moment: 1.88
CASRN: 14457-70-6, SMILES: Cl[Se]Cl, dipole_moment: 1.5206
CASRN: 14457-85-3, SMILES: B=S, dipole_moment: 1.3
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/14500-81-3/smiles
CASRN: 14500-81-3, SMILES: None, dipole_moment: 2.4853
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/14500-84-6/smiles
CASRN: 14500-84-6, SMILES: None, dipole_moment: 1.5428
CASRN: 14519-03-0, SMILES: BrNBr, dipole_moment: 1.4877
CASRN: 14519-10-9, SMILES: NBr, dipole_moment: 2.1085
CASRN: 14525-25-8, SMILES: Cl[As]=O, dipole_moment: 5.5152
CASRN: 14545-72-3, SMILES: [O-][N+](=O)OCl, dipole_moment: 0.77
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/14616-31-0/smiles
CASRN: 14616-31-0, SMILES: None, dipole_moment: 2.2595
CASRN: 14701-22-5, SMILES: [Ni++], dipole_moment: 0.0
CASRN: 14720-35-5, SMILES: BN, dipole_moment: 2.5557
CASRN: 14762-94-8, SMILES: [F], dipole_moment: 0.0
CASRN: 14808-60-7, SMILES: O=[Si]=O, dipole_moment: 1.5189
CASRN: 14885-60-0, SMILES: [3H][H], dipole_moment: 0.0
CASRN: 14915-07-2, SMILES: [O-][O-], dipole_moment: 0.0
CASRN: 14939-17-4, SMILES: [GeH3]O[GeH3], dipole_moment: 1.6282
CASRN: 14939-45-8, SMILES: [SiH3][Se][SiH3], dipole_moment: 1.5206
CASRN: 14940-63-7, SMILES: O[2H], dipole_moment: 1.85
CASRN: 14940-65-9, SMILES: O([3H])[3H], dipole_moment: 2.1792
CASRN: 14984-74-8, SMILES: FPF, dipole_moment: 2.2873
CASRN: 14986-65-3, SMILES: [F]|[GeH2]|[F], dipole_moment: 2.6549
CASRN: 14989-30-1, SMILES: [O]Cl, dipole_moment: 1.24
CASRN: 15056-34-5, SMILES: NN=N, dipole_moment: 1.4085
CASRN: 15230-48-5, SMILES: [Cl]|[GeH2]|[Cl], dipole_moment: 2.3569
CASRN: 15435-77-5, SMILES: [SiH2]=[SiH2], dipole_moment: 0.3577
CASRN: 15438-31-0, SMILES: [Fe++], dipole_moment: 0.0
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/15454-33-8/smiles
CASRN: 15454-33-8, SMILES: None, dipole_moment: 0.6548
CASRN: 15455-99-9, SMILES: [SiH3][AsH2], dipole_moment: 1.0277
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/15606-71-0/smiles
CASRN: 15606-71-0, SMILES: None, dipole_moment: 4.26
CASRN: 15626-43-4, SMILES: NN, dipole_moment: 0.0
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/15697-17-3/smiles
CASRN: 15697-17-3, SMILES: None, dipole_moment: 9.078
CASRN: 15861-05-9, SMILES: NF, dipole_moment: 2.4507
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/15905-98-3/smiles
CASRN: 15905-98-3, SMILES: None, dipole_moment: 5.2528
CASRN: 15969-58-1, SMILES: [Ti++], dipole_moment: 0.0
CASRN: 16043-45-1, SMILES: [Ti+4], dipole_moment: 0.0
CASRN: 16065-83-1, SMILES: [Cr+3], dipole_moment: 0.0
CASRN: 16136-84-8, SMILES: C\C=C/Cl, dipole_moment: 1.67
CASRN: 16136-85-9, SMILES: C/C=C/Cl, dipole_moment: 1.97
CASRN: 16355-69-4, SMILES: N(=O)[2H], dipole_moment: 2.197
CASRN: 16544-95-9, SMILES: [SiH3]S[SiH3], dipole_moment: 1.7852
CASRN: 16603-84-2, SMILES: [AlH++].[Cl-].[Cl-], dipole_moment: 2.1049
CASRN: 16643-15-5, SMILES: CS[SiH3], dipole_moment: 1.8641
CASRN: 16734-12-6, SMILES: [S-][S-], dipole_moment: 0.0
CASRN: 16812-54-7, SMILES: S.[Ni], dipole_moment: 8.5058
CASRN: 16833-27-5, SMILES: [O--], dipole_moment: 0.0
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/16860-99-4/smiles
CASRN: 16860-99-4, SMILES: None, dipole_moment: 1.03
CASRN: 16873-17-9, SMILES: [2H][H], dipole_moment: 0.0
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/16890-80-5/smiles
CASRN: 16890-80-5, SMILES: None, dipole_moment: 2.4064
CASRN: 17242-52-3, SMILES: [NH2-].[K+], dipole_moment: 1.5778
CASRN: 17256-78-9, SMILES: [H+].[F-].NO, dipole_moment: 5.1162
CASRN: 17739-47-8, SMILES: N#P, dipole_moment: 2.75
CASRN: 17778-80-2, SMILES: O, dipole_moment: 0.0
CASRN: 17778-88-0, SMILES: N, dipole_moment: 0.0
CASRN: 18282-77-4, SMILES: C=C=S, dipole_moment: 2.2255
CASRN: 18496-25-8, SMILES: [S--], dipole_moment: 0.0
CASRN: 18540-29-9, SMILES: [Cr+6], dipole_moment: 0.0
CASRN: 18820-63-8, SMILES: F[S]#N, dipole_moment: 0.7509
CASRN: 18852-54-5, SMILES: [GeH3]S[GeH3], dipole_moment: 1.6696
CASRN: 19121-56-3, SMILES: B=P, dipole_moment: 1.8912
CASRN: 19287-45-7, SMILES: BB, dipole_moment: 0.0
CASRN: 19650-79-4, SMILES: [OH-].[Cu++], dipole_moment: 1.5017
CASRN: 20074-52-6, SMILES: [Fe+3], dipole_moment: 0.0
CASRN: 20548-54-3, SMILES: S=[Ca], dipole_moment: 12.9483
CASRN: 20619-16-3, SMILES: O=[GeH2], dipole_moment: 3.28
CASRN: 20729-41-3, SMILES: CC=N, dipole_moment: 1.9
CASRN: 20859-73-8, SMILES: [Al+3].[P-3], dipole_moment: 4.2674
CASRN: 20901-21-7, SMILES: O=[S]=S, dipole_moment: 1.47
CASRN: 22400-26-6, SMILES: [N]=C=O, dipole_moment: 0.64
CASRN: 22537-15-1, SMILES: [Cl], dipole_moment: 0.0
CASRN: 22537-20-8, SMILES: [Be++], dipole_moment: 0.0
CASRN: 22537-22-0, SMILES: [Mg++], dipole_moment: 0.0
CASRN: 22541-48-6, SMILES: [Se--], dipole_moment: 0.0
CASRN: 22541-75-9, SMILES: [Ti+4], dipole_moment: 0.0
CASRN: 22541-79-3, SMILES: [Cr++], dipole_moment: 0.0
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/22755-01-7/smiles
CASRN: 22755-01-7, SMILES: None, dipole_moment: 4.5999
CASRN: 22831-39-6, SMILES: [Mg]=[Si]=[Mg], dipole_moment: 1.5712
CASRN: 22831-42-1, SMILES: [Al]#[As], dipole_moment: 1.1543
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/23058-19-7/smiles
CASRN: 23058-19-7, SMILES: None, dipole_moment: 0.0
CASRN: 23330-87-2, SMILES: [AlH2][SeH], dipole_moment: 7.1131
CASRN: 23361-55-9, SMILES: ClB=O, dipole_moment: 2.746
CASRN: 23361-56-0, SMILES: FB=O, dipole_moment: 1.852
CASRN: 23550-45-0, SMILES: S=S, dipole_moment: 0.0
CASRN: 23713-49-7, SMILES: [Zn++], dipole_moment: 0.0
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/24167-76-8/smiles
CASRN: 24167-76-8, SMILES: None, dipole_moment: 1.5977
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/24228-44-2/smiles
CASRN: 24228-44-2, SMILES: None, dipole_moment: 3.2689
CASRN: 24286-05-3, SMILES: C(=O)[2H], dipole_moment: 2.7614
CASRN: 24304-00-5, SMILES: N#[Al], dipole_moment: 2.939
CASRN: 24567-53-1, SMILES: PCl, dipole_moment: 2.3915
CASRN: 24623-77-6, SMILES: O[Al]=O, dipole_moment: 5.0947
CASRN: 24646-85-3, SMILES: N#[V], dipole_moment: 8.4895
CASRN: 24981-14-4, SMILES: CCF, dipole_moment: 1.2018
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/25277-93-4/smiles
CASRN: 25277-93-4, SMILES: None, dipole_moment: 1.9582
CASRN: 25400-21-9, SMILES: O[AsH2], dipole_moment: 2.9318
CASRN: 25617-97-4, SMILES: N#[Ga], dipole_moment: 1.6499
CASRN: 25681-79-2, SMILES: [Na][Na], dipole_moment: 0.0
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/25756-83-6/smiles
CASRN: 25756-83-6, SMILES: None, dipole_moment: 1.3996
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/25756-87-0/smiles
CASRN: 25756-87-0, SMILES: None, dipole_moment: 0.6532
CASRN: 26404-66-0, SMILES: OO[N+]([O-])=O, dipole_moment: 1.99
CASRN: 26981-93-1, SMILES: CN=N, dipole_moment: 0.3509
CASRN: 27016-73-5, SMILES: [Co]#[As], dipole_moment: 6.2613
CASRN: 29210-66-0, SMILES: [SiH3]C#N, dipole_moment: 3.3394
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/29696-97-7/smiles
CASRN: 29696-97-7, SMILES: None, dipole_moment: 2.1645
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/29904-79-8/smiles
CASRN: 29904-79-8, SMILES: None, dipole_moment: 0.0
CASRN: 30830-20-7, SMILES: C1CC2=C1CC2, dipole_moment: 0.0
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/31139-16-9/smiles
CASRN: 31139-16-9, SMILES: None, dipole_moment: 0.4714
CASRN: 31219-53-1, SMILES: O=[AsH3], dipole_moment: 5.1605
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/31479-14-8/smiles
CASRN: 31479-14-8, SMILES: None, dipole_moment: 0.2833
CASRN: 32038-79-2, SMILES: OC#C, dipole_moment: 2.2219
CASRN: 32752-94-6, SMILES: [Al]#[Al], dipole_moment: 0.0
CASRN: 34627-31-1, SMILES: CC#P, dipole_moment: 1.7659
CASRN: 35576-91-1, SMILES: NN=O, dipole_moment: 3.2031
CASRN: 36336-08-0, SMILES: COF, dipole_moment: 2.3083
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/36429-11-5/smiles
CASRN: 36429-11-5, SMILES: None, dipole_moment: 1.5987
CASRN: 36835-58-2, SMILES: [SiH3][SiH3], dipole_moment: 0.2963
CASRN: 37297-57-7, SMILES: [AlH2]C#C, dipole_moment: 2.3455
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/38063-40-0/smiles
CASRN: 38063-40-0, SMILES: None, dipole_moment: 0.3386
CASRN: 38490-24-3, SMILES: BrB=O, dipole_moment: 5.6476
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/39432-25-2/smiles
CASRN: 39432-25-2, SMILES: None, dipole_moment: 0.1437
CASRN: 39594-91-7, SMILES: SSCl, dipole_moment: 2.0027
CASRN: 40100-16-1, SMILES: C=[S]=O, dipole_moment: 1.4798
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/41591-55-3/smiles
CASRN: 41591-55-3, SMILES: None, dipole_moment: 3.2125
CASRN: 41916-72-7, SMILES: P=P, dipole_moment: 2.01
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/43435-60-5/smiles
CASRN: 43435-60-5, SMILES: None, dipole_moment: 5.0414
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/50927-81-6/smiles
CASRN: 50927-81-6, SMILES: None, dipole_moment: 1.4999
CASRN: 52323-02-1, SMILES: ClC#P, dipole_moment: 0.3027
CASRN: 52509-85-0, SMILES: Br[S]#N, dipole_moment: 1.0012
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/53750-02-0/smiles
CASRN: 53750-02-0, SMILES: None, dipole_moment: 1.2851
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/53844-93-2/smiles
CASRN: 53844-93-2, SMILES: None, dipole_moment: 1.8309
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/54098-92-9/smiles
CASRN: 54098-92-9, SMILES: None, dipole_moment: 2.0457
CASRN: 55753-38-3, SMILES: ClB=S, dipole_moment: 6.9494
CASRN: 56400-02-3, SMILES: [N-]=[N+]=S, dipole_moment: 1.4749
CASRN: 56730-08-6, SMILES: [Na+].[O-]P, dipole_moment: 4.9264
CASRN: 58436-39-8, SMILES: PC=C, dipole_moment: 2.0578
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/60635-32-7/smiles
CASRN: 60635-32-7, SMILES: None, dipole_moment: 2.0942
CASRN: 61134-37-0, SMILES: C=C=[Se], dipole_moment: 4.5091
CASRN: 61183-51-5, SMILES: ClP=C, dipole_moment: 2.6679
CASRN: 61183-53-7, SMILES: C=P, dipole_moment: 0.87
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/62064-82-8/smiles
CASRN: 62064-82-8, SMILES: None, dipole_moment: 2.3731
CASRN: 65756-42-5, SMILES: FC#P, dipole_moment: 0.5678
CASRN: 66904-99-2, SMILES: BrB=S, dipole_moment: 7.4783
CASRN: 67281-48-5, SMILES: CC=[Se], dipole_moment: 3.1617
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/95118-37-9/smiles
CASRN: 95118-37-9, SMILES: None, dipole_moment: 3.2456
CASRN: 95250-85-4, SMILES: C=[Fe], dipole_moment: 4.7633
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/105473-14-1/smiles
CASRN: 105473-14-1, SMILES: None, dipole_moment: 7.4789
CASRN: 107257-40-9, SMILES: CC=P, dipole_moment: 2.225
CASRN: 110601-75-7, SMILES: B#P, dipole_moment: 2.3795
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/112296-53-4/smiles
CASRN: 112296-53-4, SMILES: None, dipole_moment: 3.9481
HTTP error occurred: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/115410-77-0/smiles
CASRN: 115410-77-0, SMILES: None, dipole_moment: 6.8123


In [26]:
for index, row in df.iterrows():
    if 'f' in row['SMILES'].lower():
        print(row.values)

['FC=C' 1.43 2.1977589982539567 0.4025008850021998]
['FCF' 0.46 2.491954616830719 0.4698371961748457]
['CC(F)F' 2.27 2.735899447754348 0.4852071986337911]
['FC(F)=C' 1.38 2.453324550589267 0.4197965981766817]
['FC(Cl)Cl' 1.29 1.7610554652832138 0.33100275087290054]
['FC(F)Cl' 1.4 2.0394298066677305 0.3801986598506341]
['FC(F)F' 1.65 2.1581023303804243 0.4034215583950098]
['FC(F)(Br)Br' 0.66 1.649779323336218 0.29562794500879086]
['FC(F)(F)Br' 0.64 1.5892754283742065 0.28506923652253263]
['CC(F)(F)Cl' 2.14 2.588555352542354 0.43383160761775774]
['FC(Cl)(Cl)Cl' 0.45 0.9418331314482787 0.17152749768481845]
['FC(F)(Cl)Cl' 0.51 1.1628116494011507 0.21070326643122667]
['FC(F)(F)Cl' 0.5 1.1005005064368376 0.20218335592289927]
['FC(F)(F)F' 0.0 0.028413918849957698 0.009162748357346925]
['OC(=O)C(F)(F)F' 2.28 1.3236217197828657 0.25803911031864224]
['FC(F)(Cl)C(F)(F)Cl' 0.56 0.1665542808642292 0.04445604469324228]
['FC(F)(F)C(F)(F)Cl' 0.52 1.1672431483703298 0.21540593586168305]
['FC(F)(F)C(F)(